In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from scipy.stats import gaussian_kde

from utils import *
from descriptor import *
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

base_path = os.getcwd()
warnings.filterwarnings('ignore')
filename = "../data/features/soap_topology_total.csv"
data = pd.read_csv(filename,index_col=0)
data

,cif_file,topology,first_building_block,second_building_block_type,second_building_block,edge,f-chi-0-all,f-chi-1-all,f-chi-2-all,f-chi-3-all,f-Z-0-all,f-Z-1-all,f-Z-2-all,f-Z-3-all,f-I-0-all,f-I-1-all,f-I-2-all,f-I-3-all,f-T-0-all,f-T-1-all,f-T-2-all,f-T-3-all,f-S-0-all,f-S-1-all,f-S-2-all,f-S-3-all,mc-chi-0-all,mc-chi-1-all,mc-chi-2-all,mc-chi-3-all,mc-Z-0-all,mc-Z-1-all,mc-Z-2-all,mc-Z-3-all,mc-I-0-all,mc-I-1-all,mc-I-2-all,mc-I-3-all,mc-T-0-all,mc-T-1-all,mc-T-2-all,mc-T-3-all,mc-S-0-all,mc-S-1-all,mc-S-2-all,mc-S-3-all,D_mc-chi-0-all,D_mc-chi-1-all,D_mc-chi-2-all,D_mc-chi-3-all,D_mc-Z-0-all,D_mc-Z-1-all,D_mc-Z-2-all,D_mc-Z-3-all,D_mc-I-0-all,D_mc-I-1-all,D_mc-I-2-all,D_mc-I-3-all,D_mc-T-0-all,D_mc-T-1-all,D_mc-T-2-all,D_mc-T-3-all,D_mc-S-0-all,D_mc-S-1-all,D_mc-S-2-all,D_mc-S-3-all,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-I-0,f-lig-I-1,f-lig-I-2,f-lig-I-3,f-lig-T-0,f-lig-T-1,f-lig-T-2,f-lig-T-3,f-lig-S-0,f-lig-S-1,f-lig-S-2,f-lig-S-3,lc-chi-0-all,lc-chi-1-all,lc-chi-2-all,lc-chi-3-all,lc-Z-0-all,lc-Z-1-all,lc-Z-2-all,lc-Z-3-all,lc-I-0-all,lc-I-1-all,lc-I-2-all,lc-I-3-all,lc-T-0-all,lc-T-1-all,lc-T-2-all,lc-T-3-all,lc-S-0-all,lc-S-1-all,lc-S-2-all,lc-S-3-all,lc-alpha-0-all,lc-alpha-1-all,lc-alpha-2-all,lc-alpha-3-all,D_lc-chi-0-all,D_lc-chi-1-all,D_lc-chi-2-all,D_lc-chi-3-all,D_lc-Z-0-all,D_lc-Z-1-all,D_lc-Z-2-all,D_lc-Z-3-all,D_lc-I-0-all,D_lc-I-1-all,D_lc-I-2-all,D_lc-I-3-all,D_lc-T-0-all,D_lc-T-1-all,D_lc-T-2-all,D_lc-T-3-all,D_lc-S-0-all,D_lc-S-1-all,D_lc-S-2-all,D_lc-S-3-all,D_lc-alpha-0-all,D_lc-alpha-1-all,D_lc-alpha-2-all,D_lc-alpha-3-all,func-chi-0-all,func-chi-1-all,func-chi-2-all,func-chi-3-all,func-Z-0-all,func-Z-1-all,func-Z-2-all,func-Z-3-all,func-I-0-all,func-I-1-all,func-I-2-all,func-I-3-all,func-T-0-all,func-T-1-all,func-T-2-all,func-T-3-all,func-S-0-all,func-S-1-all,func-S-2-all,func-S-3-all,func-alpha-0-all,func-alpha-1-all,func-alpha-2-all,func-alpha-3-all,D_func-chi-0-all,D_func-chi-1-all,D_func-chi-2-all,D_func-chi-3-all,D_func-Z-0-all,D_func-Z-1-all,D_func-Z-2-all,D_func-Z-3-all,D_func-I-0-all,D_func-I-1-all,D_func-I-2-all,D_func-I-3-all,D_func-T-0-all,D_func-T-1-all,D_func-T-2-all,D_func-T-3-all,D_func-S-0-all,D_func-S-1-all,D_func-S-2-all,D_func-S-3-all,D_func-alpha-0-all,D_func-alpha-1-all,D_func-alpha-2-all,D_func-alpha-3-all,Di,Df,Dif,rho,VSA,GSA,VPOV,GPOV,POAV_vol_frac,PONAV_vol_frac,GPOAV,GPONAV,POAV,PONAV,KVRH,GVRH,soap0,soap1,soap2,soap3,soap4,soap5,soap6,soap7,soap8,soap9,soap10,soap11,soap12,soap13,soap14,soap15,soap16,soap17,soap18,soap19,soap20,soap21,soap22,soap23,soap24,soap25,soap26,soap27,soap28,soap29,soap30,soap31,soap32,soap33,soap34,soap35,soap36,soap37,soap38,soap39,soap40,soap41,soap42,soap43,soap44,soap45,soap46,soap47,soap48,soap49,soap50,soap51,...,soap7310,soap7311,soap7312,soap7313,soap7314,soap7315,soap7316,soap7317,soap7318,soap7319,soap7320,soap7321,soap7322,soap7323,soap7324,soap7325,soap7326,soap7327,soap7328,soap7329,soap7330,soap7331,soap7332,soap7333,soap7334,soap7335,soap7336,soap7337,soap7338,soap7339,soap7340,soap7341,soap7342,soap7343,soap7344,soap7345,soap7346,soap7347,soap7348,soap7349,soap7350,soap7351,soap7352,soap7353,soap7354,soap7355,soap7356,soap7357,soap7358,soap7359,soap7360,soap7361,soap7362,soap7363,soap7364,soap7365,soap7366,soap7367,soap7368,soap7369,soap7370,soap7371,soap7372,soap7373,soap7374,soap7375,soap7376,soap7377,soap7378,soap7379,soap7380,soap7381,soap7382,soap7383,soap7384,soap7385,soap7386,soap7387,soap7388,soap7389,soap7390,soap7391,soap7392,soap7393,soap7394,soap7395,soap7396,soap7397,soap7398,soap7399,soap7400,soap7401,soap7402,soap7403,soap7404,soap7405,soap7406,soap7407,soap7408,soap7409,soap7410,soap7411,soap7412,soap7413,soap7414,soap7415,soap7416,soap7417,soap7418,soap7419,soap7420,soap7421,soap7422,soap7423,soap7424,soap7425,soap7426,soap7427,soap7428,soap7429,soap7430,soap7431,soap7432,soap7433,soap7434,soap7435,soap7436,soap7437,soap7438,soap7439,soap7440,soap7441,soap7442,

In [2]:
soap_columns = list(data.columns)[198:]

columns = geometric + RAC + soap_columns + property_
new_columns = geometric + RAC + soap_columns
descriptors = data[new_columns]

drop_col = remove_constant_value_features(descriptors)
new_df_columns = [e for e in descriptors.columns if e not in drop_col]

property_ = ["GVRH"]

X = data[new_df_columns]
y = data[property_]

In [3]:
train_size = 1000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)



Learning rate set to 0.040943
0:	learn: 3.3964893	total: 140ms	remaining: 2m 20s
1:	learn: 3.3458069	total: 199ms	remaining: 1m 39s
2:	learn: 3.2904445	total: 257ms	remaining: 1m 25s
3:	learn: 3.2412310	total: 315ms	remaining: 1m 18s
4:	learn: 3.2021569	total: 373ms	remaining: 1m 14s
5:	learn: 3.1535750	total: 432ms	remaining: 1m 11s
6:	learn: 3.1034413	total: 490ms	remaining: 1m 9s
7:	learn: 3.0652422	total: 550ms	remaining: 1m 8s
8:	learn: 3.0228298	total: 608ms	remaining: 1m 6s
9:	learn: 2.9796886	total: 666ms	remaining: 1m 5s
10:	learn: 2.9309588	total: 725ms	remaining: 1m 5s
11:	learn: 2.8930721	total: 783ms	remaining: 1m 4s
12:	learn: 2.8586360	total: 841ms	remaining: 1m 3s
13:	learn: 2.8187629	total: 901ms	remaining: 1m 3s
14:	learn: 2.7841665	total: 959ms	remaining: 1m 2s
15:	learn: 2.7550096	total: 1.02s	remaining: 1m 2s
16:	learn: 2.7183882	total: 1.07s	remaining: 1m 2s
17:	learn: 2.6914388	total: 1.13s	remaining: 1m 1s
18:	learn: 2.6634862	total: 1.19s	remaining: 1m 1s
19:	l

161:	learn: 1.3455352	total: 9.55s	remaining: 49.4s
162:	learn: 1.3399408	total: 9.61s	remaining: 49.3s
163:	learn: 1.3349165	total: 9.67s	remaining: 49.3s
164:	learn: 1.3303639	total: 9.73s	remaining: 49.2s
165:	learn: 1.3286327	total: 9.79s	remaining: 49.2s
166:	learn: 1.3230251	total: 9.84s	remaining: 49.1s
167:	learn: 1.3207797	total: 9.9s	remaining: 49s
168:	learn: 1.3194769	total: 9.96s	remaining: 49s
169:	learn: 1.3178828	total: 10s	remaining: 48.9s
170:	learn: 1.3167739	total: 10.1s	remaining: 48.8s
171:	learn: 1.3118471	total: 10.1s	remaining: 48.8s
172:	learn: 1.3093287	total: 10.2s	remaining: 48.7s
173:	learn: 1.3041011	total: 10.2s	remaining: 48.7s
174:	learn: 1.2991828	total: 10.3s	remaining: 48.6s
175:	learn: 1.2976985	total: 10.4s	remaining: 48.5s
176:	learn: 1.2924089	total: 10.4s	remaining: 48.5s
177:	learn: 1.2870731	total: 10.5s	remaining: 48.4s
178:	learn: 1.2819391	total: 10.5s	remaining: 48.3s
179:	learn: 1.2797314	total: 10.6s	remaining: 48.3s
180:	learn: 1.27762

321:	learn: 0.8298141	total: 18.9s	remaining: 39.8s
322:	learn: 0.8267904	total: 18.9s	remaining: 39.7s
323:	learn: 0.8233456	total: 19s	remaining: 39.7s
324:	learn: 0.8221524	total: 19.1s	remaining: 39.6s
325:	learn: 0.8184146	total: 19.1s	remaining: 39.5s
326:	learn: 0.8154979	total: 19.2s	remaining: 39.5s
327:	learn: 0.8106726	total: 19.2s	remaining: 39.4s
328:	learn: 0.8099854	total: 19.3s	remaining: 39.4s
329:	learn: 0.8067896	total: 19.4s	remaining: 39.3s
330:	learn: 0.8061325	total: 19.4s	remaining: 39.2s
331:	learn: 0.8032294	total: 19.5s	remaining: 39.2s
332:	learn: 0.8007453	total: 19.5s	remaining: 39.1s
333:	learn: 0.8000501	total: 19.6s	remaining: 39.1s
334:	learn: 0.7990799	total: 19.6s	remaining: 39s
335:	learn: 0.7962903	total: 19.7s	remaining: 38.9s
336:	learn: 0.7924352	total: 19.8s	remaining: 38.9s
337:	learn: 0.7914731	total: 19.8s	remaining: 38.8s
338:	learn: 0.7887825	total: 19.9s	remaining: 38.8s
339:	learn: 0.7863461	total: 19.9s	remaining: 38.7s
340:	learn: 0.78

481:	learn: 0.5500671	total: 28.2s	remaining: 30.3s
482:	learn: 0.5474717	total: 28.3s	remaining: 30.3s
483:	learn: 0.5452608	total: 28.3s	remaining: 30.2s
484:	learn: 0.5449368	total: 28.4s	remaining: 30.1s
485:	learn: 0.5419633	total: 28.4s	remaining: 30.1s
486:	learn: 0.5406239	total: 28.5s	remaining: 30s
487:	learn: 0.5394613	total: 28.6s	remaining: 30s
488:	learn: 0.5388340	total: 28.6s	remaining: 29.9s
489:	learn: 0.5375259	total: 28.7s	remaining: 29.8s
490:	learn: 0.5362466	total: 28.7s	remaining: 29.8s
491:	learn: 0.5355848	total: 28.8s	remaining: 29.7s
492:	learn: 0.5327030	total: 28.8s	remaining: 29.7s
493:	learn: 0.5308654	total: 28.9s	remaining: 29.6s
494:	learn: 0.5291576	total: 29s	remaining: 29.5s
495:	learn: 0.5288068	total: 29s	remaining: 29.5s
496:	learn: 0.5271741	total: 29.1s	remaining: 29.4s
497:	learn: 0.5255144	total: 29.1s	remaining: 29.4s
498:	learn: 0.5240053	total: 29.2s	remaining: 29.3s
499:	learn: 0.5236245	total: 29.3s	remaining: 29.3s
500:	learn: 0.523161

641:	learn: 0.3765599	total: 37.5s	remaining: 20.9s
642:	learn: 0.3753744	total: 37.6s	remaining: 20.9s
643:	learn: 0.3735672	total: 37.6s	remaining: 20.8s
644:	learn: 0.3733506	total: 37.7s	remaining: 20.8s
645:	learn: 0.3723378	total: 37.8s	remaining: 20.7s
646:	learn: 0.3712314	total: 37.8s	remaining: 20.6s
647:	learn: 0.3709547	total: 37.9s	remaining: 20.6s
648:	learn: 0.3699591	total: 37.9s	remaining: 20.5s
649:	learn: 0.3688709	total: 38s	remaining: 20.5s
650:	learn: 0.3672008	total: 38.1s	remaining: 20.4s
651:	learn: 0.3664872	total: 38.1s	remaining: 20.3s
652:	learn: 0.3653960	total: 38.2s	remaining: 20.3s
653:	learn: 0.3648809	total: 38.2s	remaining: 20.2s
654:	learn: 0.3646670	total: 38.3s	remaining: 20.2s
655:	learn: 0.3634400	total: 38.3s	remaining: 20.1s
656:	learn: 0.3632468	total: 38.4s	remaining: 20s
657:	learn: 0.3615146	total: 38.5s	remaining: 20s
658:	learn: 0.3605897	total: 38.5s	remaining: 19.9s
659:	learn: 0.3596033	total: 38.6s	remaining: 19.9s
660:	learn: 0.3589

...

331:	learn: 0.8443600	total: 19.5s	remaining: 39.2s
332:	learn: 0.8433254	total: 19.5s	remaining: 39.1s
333:	learn: 0.8401346	total: 19.6s	remaining: 39.1s
334:	learn: 0.8380121	total: 19.7s	remaining: 39s
335:	learn: 0.8337102	total: 19.7s	remaining: 39s
336:	learn: 0.8302092	total: 19.8s	remaining: 38.9s
337:	learn: 0.8261335	total: 19.8s	remaining: 38.8s
338:	learn: 0.8253738	total: 19.9s	remaining: 38.8s
339:	learn: 0.8218509	total: 19.9s	remaining: 38.7s
340:	learn: 0.8192384	total: 20s	remaining: 38.7s
341:	learn: 0.8155602	total: 20.1s	remaining: 38.6s
342:	learn: 0.8143341	total: 20.1s	remaining: 38.5s
343:	learn: 0.8128738	total: 20.2s	remaining: 38.5s
344:	learn: 0.8119924	total: 20.2s	remaining: 38.4s
345:	learn: 0.8097432	total: 20.3s	remaining: 38.4s
346:	learn: 0.8088524	total: 20.4s	remaining: 38.3s
347:	learn: 0.8052043	total: 20.4s	remaining: 38.3s
348:	learn: 0.8042131	total: 20.5s	remaining: 38.2s
349:	learn: 0.8016046	total: 20.5s	remaining: 38.1s
350:	learn: 0.8008

491:	learn: 0.5518237	total: 28.9s	remaining: 29.8s
492:	learn: 0.5496714	total: 28.9s	remaining: 29.8s
493:	learn: 0.5491224	total: 29s	remaining: 29.7s
494:	learn: 0.5470651	total: 29.1s	remaining: 29.6s
495:	learn: 0.5455760	total: 29.1s	remaining: 29.6s
496:	learn: 0.5451656	total: 29.2s	remaining: 29.5s
497:	learn: 0.5424753	total: 29.2s	remaining: 29.5s
498:	learn: 0.5406271	total: 29.3s	remaining: 29.4s
499:	learn: 0.5382470	total: 29.4s	remaining: 29.4s
500:	learn: 0.5367464	total: 29.4s	remaining: 29.3s
501:	learn: 0.5358340	total: 29.5s	remaining: 29.2s
502:	learn: 0.5336569	total: 29.5s	remaining: 29.2s
503:	learn: 0.5329672	total: 29.6s	remaining: 29.1s
504:	learn: 0.5310226	total: 29.6s	remaining: 29.1s
505:	learn: 0.5294083	total: 29.7s	remaining: 29s
506:	learn: 0.5287582	total: 29.8s	remaining: 28.9s
507:	learn: 0.5271757	total: 29.8s	remaining: 28.9s
508:	learn: 0.5253143	total: 29.9s	remaining: 28.8s
509:	learn: 0.5239493	total: 29.9s	remaining: 28.8s
510:	learn: 0.52

651:	learn: 0.3702478	total: 38.3s	remaining: 20.4s
652:	learn: 0.3683269	total: 38.3s	remaining: 20.4s
653:	learn: 0.3677899	total: 38.4s	remaining: 20.3s
654:	learn: 0.3673219	total: 38.5s	remaining: 20.3s
655:	learn: 0.3656364	total: 38.5s	remaining: 20.2s
656:	learn: 0.3640779	total: 38.6s	remaining: 20.1s
657:	learn: 0.3632814	total: 38.6s	remaining: 20.1s
658:	learn: 0.3617318	total: 38.7s	remaining: 20s
659:	learn: 0.3605445	total: 38.8s	remaining: 20s
660:	learn: 0.3596327	total: 38.8s	remaining: 19.9s
661:	learn: 0.3583024	total: 38.9s	remaining: 19.9s
662:	learn: 0.3580979	total: 38.9s	remaining: 19.8s
663:	learn: 0.3578592	total: 39s	remaining: 19.7s
664:	learn: 0.3571493	total: 39.1s	remaining: 19.7s
665:	learn: 0.3563884	total: 39.1s	remaining: 19.6s
666:	learn: 0.3555864	total: 39.2s	remaining: 19.6s
667:	learn: 0.3546632	total: 39.2s	remaining: 19.5s
668:	learn: 0.3535841	total: 39.3s	remaining: 19.4s
669:	learn: 0.3533633	total: 39.3s	remaining: 19.4s
670:	learn: 0.3529

811:	learn: 0.2544776	total: 47.7s	remaining: 11.1s
812:	learn: 0.2537074	total: 47.8s	remaining: 11s
813:	learn: 0.2525734	total: 47.9s	remaining: 10.9s
814:	learn: 0.2515255	total: 47.9s	remaining: 10.9s
815:	learn: 0.2505909	total: 48s	remaining: 10.8s
816:	learn: 0.2500361	total: 48s	remaining: 10.8s
817:	learn: 0.2493459	total: 48.1s	remaining: 10.7s
818:	learn: 0.2482269	total: 48.1s	remaining: 10.6s
819:	learn: 0.2477367	total: 48.2s	remaining: 10.6s
820:	learn: 0.2469548	total: 48.3s	remaining: 10.5s
821:	learn: 0.2459636	total: 48.3s	remaining: 10.5s
822:	learn: 0.2456335	total: 48.4s	remaining: 10.4s
823:	learn: 0.2451878	total: 48.4s	remaining: 10.3s
824:	learn: 0.2445261	total: 48.5s	remaining: 10.3s
825:	learn: 0.2436677	total: 48.6s	remaining: 10.2s
826:	learn: 0.2430495	total: 48.6s	remaining: 10.2s
827:	learn: 0.2425870	total: 48.7s	remaining: 10.1s
828:	learn: 0.2419072	total: 48.7s	remaining: 10.1s
829:	learn: 0.2411169	total: 48.8s	remaining: 9.99s
830:	learn: 0.2402

971:	learn: 0.1774863	total: 57.2s	remaining: 1.65s
972:	learn: 0.1767501	total: 57.2s	remaining: 1.59s
973:	learn: 0.1762665	total: 57.3s	remaining: 1.53s
974:	learn: 0.1761055	total: 57.3s	remaining: 1.47s
975:	learn: 0.1757720	total: 57.4s	remaining: 1.41s
976:	learn: 0.1753630	total: 57.4s	remaining: 1.35s
977:	learn: 0.1748362	total: 57.5s	remaining: 1.29s
978:	learn: 0.1746186	total: 57.6s	remaining: 1.23s
979:	learn: 0.1743915	total: 57.6s	remaining: 1.18s
980:	learn: 0.1742408	total: 57.7s	remaining: 1.12s
981:	learn: 0.1737436	total: 57.7s	remaining: 1.06s
982:	learn: 0.1731050	total: 57.8s	remaining: 1000ms
983:	learn: 0.1725376	total: 57.9s	remaining: 941ms
984:	learn: 0.1723896	total: 57.9s	remaining: 882ms
985:	learn: 0.1719594	total: 58s	remaining: 823ms
986:	learn: 0.1715493	total: 58s	remaining: 764ms
987:	learn: 0.1711387	total: 58.1s	remaining: 706ms
988:	learn: 0.1708078	total: 58.2s	remaining: 647ms
989:	learn: 0.1706423	total: 58.2s	remaining: 588ms
990:	learn: 0.1

In [4]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.0305404743074964
1.0285714285224326
0.9957014583967736
1.092906503941879
1.0020787718202078
1.0334442088561897
0.503120925792703
0.4800808855550097
0.5157044045787995
0.49530477245446525
0.4956004272743787
0.5289141391008618


In [5]:
train_size = 2000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)


Learning rate set to 0.045682
0:	learn: 3.1430491	total: 112ms	remaining: 1m 52s
1:	learn: 3.1046955	total: 192ms	remaining: 1m 35s
2:	learn: 3.0609736	total: 271ms	remaining: 1m 30s
3:	learn: 3.0202971	total: 351ms	remaining: 1m 27s
4:	learn: 2.9775956	total: 430ms	remaining: 1m 25s
5:	learn: 2.9368484	total: 508ms	remaining: 1m 24s
6:	learn: 2.9056717	total: 587ms	remaining: 1m 23s
7:	learn: 2.8727252	total: 666ms	remaining: 1m 22s
8:	learn: 2.8395164	total: 745ms	remaining: 1m 22s
9:	learn: 2.8079668	total: 824ms	remaining: 1m 21s
10:	learn: 2.7711470	total: 903ms	remaining: 1m 21s
11:	learn: 2.7468852	total: 983ms	remaining: 1m 20s
12:	learn: 2.7204900	total: 1.06s	remaining: 1m 20s
13:	learn: 2.6950215	total: 1.14s	remaining: 1m 20s
14:	learn: 2.6692493	total: 1.22s	remaining: 1m 20s
15:	learn: 2.6456149	total: 1.3s	remaining: 1m 19s
16:	learn: 2.6256968	total: 1.38s	remaining: 1m 19s
17:	learn: 2.6066817	total: 1.46s	remaining: 1m 19s
18:	learn: 2.5838385	total: 1.54s	remaining: 

160:	learn: 1.6168983	total: 12.7s	remaining: 1m 6s
161:	learn: 1.6085281	total: 12.8s	remaining: 1m 6s
162:	learn: 1.6064232	total: 12.9s	remaining: 1m 6s
163:	learn: 1.6011724	total: 13s	remaining: 1m 6s
164:	learn: 1.5985666	total: 13.1s	remaining: 1m 6s
165:	learn: 1.5938652	total: 13.1s	remaining: 1m 6s
166:	learn: 1.5909042	total: 13.2s	remaining: 1m 5s
167:	learn: 1.5892513	total: 13.3s	remaining: 1m 5s
168:	learn: 1.5830006	total: 13.4s	remaining: 1m 5s
169:	learn: 1.5757612	total: 13.5s	remaining: 1m 5s
170:	learn: 1.5664250	total: 13.5s	remaining: 1m 5s
171:	learn: 1.5632055	total: 13.6s	remaining: 1m 5s
172:	learn: 1.5603049	total: 13.7s	remaining: 1m 5s
173:	learn: 1.5543910	total: 13.8s	remaining: 1m 5s
174:	learn: 1.5514661	total: 13.9s	remaining: 1m 5s
175:	learn: 1.5499108	total: 13.9s	remaining: 1m 5s
176:	learn: 1.5479126	total: 14s	remaining: 1m 5s
177:	learn: 1.5458905	total: 14.1s	remaining: 1m 5s
178:	learn: 1.5443101	total: 14.2s	remaining: 1m 4s
179:	learn: 1.53

322:	learn: 1.0530109	total: 25.5s	remaining: 53.5s
323:	learn: 1.0494901	total: 25.6s	remaining: 53.4s
324:	learn: 1.0476947	total: 25.7s	remaining: 53.4s
325:	learn: 1.0430652	total: 25.8s	remaining: 53.3s
326:	learn: 1.0413463	total: 25.9s	remaining: 53.2s
327:	learn: 1.0387716	total: 25.9s	remaining: 53.1s
328:	learn: 1.0350654	total: 26s	remaining: 53.1s
329:	learn: 1.0342943	total: 26.1s	remaining: 53s
330:	learn: 1.0315990	total: 26.2s	remaining: 52.9s
331:	learn: 1.0292960	total: 26.2s	remaining: 52.8s
332:	learn: 1.0263723	total: 26.3s	remaining: 52.7s
333:	learn: 1.0256201	total: 26.4s	remaining: 52.7s
334:	learn: 1.0245859	total: 26.5s	remaining: 52.6s
335:	learn: 1.0215125	total: 26.6s	remaining: 52.5s
336:	learn: 1.0186341	total: 26.6s	remaining: 52.4s
337:	learn: 1.0156098	total: 26.7s	remaining: 52.3s
338:	learn: 1.0110695	total: 26.8s	remaining: 52.3s
339:	learn: 1.0090256	total: 26.9s	remaining: 52.2s
340:	learn: 1.0063451	total: 27s	remaining: 52.1s
341:	learn: 1.0056

481:	learn: 0.7586648	total: 38.1s	remaining: 40.9s
482:	learn: 0.7568699	total: 38.2s	remaining: 40.9s
483:	learn: 0.7555042	total: 38.2s	remaining: 40.8s
484:	learn: 0.7543156	total: 38.3s	remaining: 40.7s
485:	learn: 0.7518031	total: 38.4s	remaining: 40.6s
486:	learn: 0.7514286	total: 38.5s	remaining: 40.5s
487:	learn: 0.7489773	total: 38.6s	remaining: 40.5s
488:	learn: 0.7475742	total: 38.6s	remaining: 40.4s
489:	learn: 0.7472249	total: 38.7s	remaining: 40.3s
490:	learn: 0.7454643	total: 38.8s	remaining: 40.2s
491:	learn: 0.7434830	total: 38.9s	remaining: 40.1s
492:	learn: 0.7419983	total: 39s	remaining: 40.1s
493:	learn: 0.7416175	total: 39s	remaining: 40s
494:	learn: 0.7397443	total: 39.1s	remaining: 39.9s
495:	learn: 0.7379317	total: 39.2s	remaining: 39.8s
496:	learn: 0.7363790	total: 39.3s	remaining: 39.7s
497:	learn: 0.7349406	total: 39.4s	remaining: 39.7s
498:	learn: 0.7321335	total: 39.4s	remaining: 39.6s
499:	learn: 0.7317892	total: 39.5s	remaining: 39.5s
500:	learn: 0.7304

640:	learn: 0.5705936	total: 50.7s	remaining: 28.4s
641:	learn: 0.5700665	total: 50.7s	remaining: 28.3s
642:	learn: 0.5688799	total: 50.8s	remaining: 28.2s
643:	learn: 0.5674904	total: 50.9s	remaining: 28.1s
644:	learn: 0.5672495	total: 51s	remaining: 28.1s
645:	learn: 0.5658054	total: 51.1s	remaining: 28s
646:	learn: 0.5645256	total: 51.1s	remaining: 27.9s
647:	learn: 0.5642887	total: 51.2s	remaining: 27.8s
648:	learn: 0.5631637	total: 51.3s	remaining: 27.7s
649:	learn: 0.5629194	total: 51.4s	remaining: 27.7s
650:	learn: 0.5627187	total: 51.5s	remaining: 27.6s
651:	learn: 0.5610587	total: 51.5s	remaining: 27.5s
652:	learn: 0.5595638	total: 51.6s	remaining: 27.4s
653:	learn: 0.5583193	total: 51.7s	remaining: 27.3s
654:	learn: 0.5570539	total: 51.8s	remaining: 27.3s
655:	learn: 0.5564391	total: 51.9s	remaining: 27.2s
656:	learn: 0.5550440	total: 51.9s	remaining: 27.1s
657:	learn: 0.5544315	total: 52s	remaining: 27s
658:	learn: 0.5537070	total: 52.1s	remaining: 27s
659:	learn: 0.5524848	

...

302:	learn: 1.0221522	total: 24s	remaining: 55.1s
303:	learn: 1.0189530	total: 24s	remaining: 55.1s
304:	learn: 1.0178798	total: 24.1s	remaining: 55s
305:	learn: 1.0136356	total: 24.2s	remaining: 54.9s
306:	learn: 1.0105184	total: 24.3s	remaining: 54.8s
307:	learn: 1.0098182	total: 24.4s	remaining: 54.7s
308:	learn: 1.0064118	total: 24.4s	remaining: 54.7s
309:	learn: 1.0057752	total: 24.5s	remaining: 54.6s
310:	learn: 1.0046299	total: 24.6s	remaining: 54.5s
311:	learn: 1.0039536	total: 24.7s	remaining: 54.4s
312:	learn: 1.0019679	total: 24.8s	remaining: 54.3s
313:	learn: 0.9985852	total: 24.8s	remaining: 54.3s
314:	learn: 0.9963365	total: 24.9s	remaining: 54.2s
315:	learn: 0.9938547	total: 25s	remaining: 54.1s
316:	learn: 0.9909045	total: 25.1s	remaining: 54s
317:	learn: 0.9902331	total: 25.1s	remaining: 53.9s
318:	learn: 0.9895765	total: 25.2s	remaining: 53.9s
319:	learn: 0.9870989	total: 25.3s	remaining: 53.8s
320:	learn: 0.9848348	total: 25.4s	remaining: 53.7s
321:	learn: 0.9834936	

461:	learn: 0.7545973	total: 36.5s	remaining: 42.6s
462:	learn: 0.7529688	total: 36.6s	remaining: 42.5s
463:	learn: 0.7509482	total: 36.7s	remaining: 42.4s
464:	learn: 0.7488164	total: 36.8s	remaining: 42.3s
465:	learn: 0.7470694	total: 36.9s	remaining: 42.2s
466:	learn: 0.7457893	total: 36.9s	remaining: 42.2s
467:	learn: 0.7448488	total: 37s	remaining: 42.1s
468:	learn: 0.7423722	total: 37.1s	remaining: 42s
469:	learn: 0.7390977	total: 37.2s	remaining: 41.9s
470:	learn: 0.7378182	total: 37.3s	remaining: 41.8s
471:	learn: 0.7361889	total: 37.3s	remaining: 41.8s
472:	learn: 0.7356493	total: 37.4s	remaining: 41.7s
473:	learn: 0.7330791	total: 37.5s	remaining: 41.6s
474:	learn: 0.7313890	total: 37.6s	remaining: 41.5s
475:	learn: 0.7308327	total: 37.7s	remaining: 41.4s
476:	learn: 0.7289395	total: 37.7s	remaining: 41.4s
477:	learn: 0.7277619	total: 37.8s	remaining: 41.3s
478:	learn: 0.7254658	total: 37.9s	remaining: 41.2s
479:	learn: 0.7244512	total: 38s	remaining: 41.1s
480:	learn: 0.7237

620:	learn: 0.5716772	total: 49.1s	remaining: 30s
621:	learn: 0.5707653	total: 49.2s	remaining: 29.9s
622:	learn: 0.5693502	total: 49.3s	remaining: 29.8s
623:	learn: 0.5679724	total: 49.4s	remaining: 29.7s
624:	learn: 0.5670604	total: 49.4s	remaining: 29.7s
625:	learn: 0.5655205	total: 49.5s	remaining: 29.6s
626:	learn: 0.5642038	total: 49.6s	remaining: 29.5s
627:	learn: 0.5637685	total: 49.7s	remaining: 29.4s
628:	learn: 0.5635445	total: 49.8s	remaining: 29.4s
629:	learn: 0.5618822	total: 49.8s	remaining: 29.3s
630:	learn: 0.5607297	total: 49.9s	remaining: 29.2s
631:	learn: 0.5596362	total: 50s	remaining: 29.1s
632:	learn: 0.5587683	total: 50.1s	remaining: 29s
633:	learn: 0.5583373	total: 50.2s	remaining: 29s
634:	learn: 0.5572655	total: 50.2s	remaining: 28.9s
635:	learn: 0.5560871	total: 50.3s	remaining: 28.8s
636:	learn: 0.5550869	total: 50.4s	remaining: 28.7s
637:	learn: 0.5548675	total: 50.5s	remaining: 28.6s
638:	learn: 0.5543793	total: 50.6s	remaining: 28.6s
639:	learn: 0.553578

782:	learn: 0.4400461	total: 1m 1s	remaining: 17.2s
783:	learn: 0.4390002	total: 1m 2s	remaining: 17.1s
784:	learn: 0.4384732	total: 1m 2s	remaining: 17s
785:	learn: 0.4378141	total: 1m 2s	remaining: 16.9s
786:	learn: 0.4375370	total: 1m 2s	remaining: 16.9s
787:	learn: 0.4371388	total: 1m 2s	remaining: 16.8s
788:	learn: 0.4362622	total: 1m 2s	remaining: 16.7s
789:	learn: 0.4354405	total: 1m 2s	remaining: 16.6s
790:	learn: 0.4352766	total: 1m 2s	remaining: 16.5s
791:	learn: 0.4349318	total: 1m 2s	remaining: 16.5s
792:	learn: 0.4340857	total: 1m 2s	remaining: 16.4s
793:	learn: 0.4331837	total: 1m 2s	remaining: 16.3s
794:	learn: 0.4322358	total: 1m 2s	remaining: 16.2s
795:	learn: 0.4316345	total: 1m 3s	remaining: 16.1s
796:	learn: 0.4311227	total: 1m 3s	remaining: 16.1s
797:	learn: 0.4301562	total: 1m 3s	remaining: 16s
798:	learn: 0.4296442	total: 1m 3s	remaining: 15.9s
799:	learn: 0.4292696	total: 1m 3s	remaining: 15.8s
800:	learn: 0.4288329	total: 1m 3s	remaining: 15.8s
801:	learn: 0.42

941:	learn: 0.3489597	total: 1m 14s	remaining: 4.59s
942:	learn: 0.3481326	total: 1m 14s	remaining: 4.51s
943:	learn: 0.3479376	total: 1m 14s	remaining: 4.43s
944:	learn: 0.3476830	total: 1m 14s	remaining: 4.36s
945:	learn: 0.3469015	total: 1m 14s	remaining: 4.28s
946:	learn: 0.3465620	total: 1m 15s	remaining: 4.2s
947:	learn: 0.3463747	total: 1m 15s	remaining: 4.12s
948:	learn: 0.3456060	total: 1m 15s	remaining: 4.04s
949:	learn: 0.3449347	total: 1m 15s	remaining: 3.96s
950:	learn: 0.3442308	total: 1m 15s	remaining: 3.88s
951:	learn: 0.3433657	total: 1m 15s	remaining: 3.8s
952:	learn: 0.3427755	total: 1m 15s	remaining: 3.72s
953:	learn: 0.3423292	total: 1m 15s	remaining: 3.64s
954:	learn: 0.3418337	total: 1m 15s	remaining: 3.56s
955:	learn: 0.3416338	total: 1m 15s	remaining: 3.48s
956:	learn: 0.3412238	total: 1m 15s	remaining: 3.4s
957:	learn: 0.3408669	total: 1m 15s	remaining: 3.33s
958:	learn: 0.3401219	total: 1m 15s	remaining: 3.25s
959:	learn: 0.3394603	total: 1m 16s	remaining: 3.

In [6]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.9520108274624441
0.9697635720782501
0.9319587136658952
0.9929610073432165
0.910696028052217
0.9546748161726414
0.5508604203200992
0.5278888448995924
0.5507966010613343
0.5447728324499759
0.5600929201535718
0.5707509030360215


In [7]:
train_size = 3000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)


Learning rate set to 0.048704
0:	learn: 3.1944544	total: 118ms	remaining: 1m 57s
1:	learn: 3.1466929	total: 212ms	remaining: 1m 45s
2:	learn: 3.1043534	total: 306ms	remaining: 1m 41s
3:	learn: 3.0630527	total: 400ms	remaining: 1m 39s
4:	learn: 3.0270627	total: 494ms	remaining: 1m 38s
5:	learn: 2.9920104	total: 588ms	remaining: 1m 37s
6:	learn: 2.9550467	total: 682ms	remaining: 1m 36s
7:	learn: 2.9203285	total: 776ms	remaining: 1m 36s
8:	learn: 2.8851359	total: 871ms	remaining: 1m 35s
9:	learn: 2.8543405	total: 966ms	remaining: 1m 35s
10:	learn: 2.8192464	total: 1.06s	remaining: 1m 35s
11:	learn: 2.7951136	total: 1.15s	remaining: 1m 35s
12:	learn: 2.7654629	total: 1.25s	remaining: 1m 34s
13:	learn: 2.7349620	total: 1.34s	remaining: 1m 34s
14:	learn: 2.7055318	total: 1.44s	remaining: 1m 34s
15:	learn: 2.6781486	total: 1.53s	remaining: 1m 34s
16:	learn: 2.6537413	total: 1.63s	remaining: 1m 34s
17:	learn: 2.6303669	total: 1.72s	remaining: 1m 33s
18:	learn: 2.6048774	total: 1.82s	remaining:

159:	learn: 1.7165437	total: 15.1s	remaining: 1m 19s
160:	learn: 1.7115516	total: 15.2s	remaining: 1m 18s
161:	learn: 1.7077801	total: 15.2s	remaining: 1m 18s
162:	learn: 1.7031335	total: 15.3s	remaining: 1m 18s
163:	learn: 1.7016888	total: 15.4s	remaining: 1m 18s
164:	learn: 1.7001179	total: 15.5s	remaining: 1m 18s
165:	learn: 1.6972042	total: 15.6s	remaining: 1m 18s
166:	learn: 1.6930648	total: 15.7s	remaining: 1m 18s
167:	learn: 1.6883090	total: 15.8s	remaining: 1m 18s
168:	learn: 1.6844636	total: 15.9s	remaining: 1m 18s
169:	learn: 1.6825865	total: 16s	remaining: 1m 18s
170:	learn: 1.6752557	total: 16.1s	remaining: 1m 18s
171:	learn: 1.6736246	total: 16.2s	remaining: 1m 17s
172:	learn: 1.6715118	total: 16.3s	remaining: 1m 17s
173:	learn: 1.6653355	total: 16.4s	remaining: 1m 17s
174:	learn: 1.6602854	total: 16.5s	remaining: 1m 17s
175:	learn: 1.6588722	total: 16.6s	remaining: 1m 17s
176:	learn: 1.6541386	total: 16.6s	remaining: 1m 17s
177:	learn: 1.6524478	total: 16.7s	remaining: 1m

318:	learn: 1.2208973	total: 30s	remaining: 1m 3s
319:	learn: 1.2159676	total: 30.1s	remaining: 1m 3s
320:	learn: 1.2153175	total: 30.2s	remaining: 1m 3s
321:	learn: 1.2123616	total: 30.3s	remaining: 1m 3s
322:	learn: 1.2090094	total: 30.4s	remaining: 1m 3s
323:	learn: 1.2079169	total: 30.5s	remaining: 1m 3s
324:	learn: 1.2054387	total: 30.6s	remaining: 1m 3s
325:	learn: 1.2027867	total: 30.6s	remaining: 1m 3s
326:	learn: 1.2001436	total: 30.7s	remaining: 1m 3s
327:	learn: 1.1995185	total: 30.8s	remaining: 1m 3s
328:	learn: 1.1969632	total: 30.9s	remaining: 1m 3s
329:	learn: 1.1956037	total: 31s	remaining: 1m 2s
330:	learn: 1.1949721	total: 31.1s	remaining: 1m 2s
331:	learn: 1.1919624	total: 31.2s	remaining: 1m 2s
332:	learn: 1.1906915	total: 31.3s	remaining: 1m 2s
333:	learn: 1.1880509	total: 31.4s	remaining: 1m 2s
334:	learn: 1.1870747	total: 31.5s	remaining: 1m 2s
335:	learn: 1.1865036	total: 31.6s	remaining: 1m 2s
336:	learn: 1.1831676	total: 31.7s	remaining: 1m 2s
337:	learn: 1.18

478:	learn: 0.9280199	total: 45s	remaining: 49s
479:	learn: 0.9261395	total: 45.1s	remaining: 48.9s
480:	learn: 0.9244336	total: 45.2s	remaining: 48.8s
481:	learn: 0.9237113	total: 45.3s	remaining: 48.7s
482:	learn: 0.9231359	total: 45.4s	remaining: 48.6s
483:	learn: 0.9225966	total: 45.5s	remaining: 48.5s
484:	learn: 0.9222697	total: 45.6s	remaining: 48.4s
485:	learn: 0.9197583	total: 45.7s	remaining: 48.3s
486:	learn: 0.9185211	total: 45.8s	remaining: 48.2s
487:	learn: 0.9178413	total: 45.9s	remaining: 48.1s
488:	learn: 0.9159918	total: 46s	remaining: 48s
489:	learn: 0.9137576	total: 46.1s	remaining: 48s
490:	learn: 0.9117744	total: 46.2s	remaining: 47.9s
491:	learn: 0.9112336	total: 46.3s	remaining: 47.8s
492:	learn: 0.9100377	total: 46.4s	remaining: 47.7s
493:	learn: 0.9097111	total: 46.4s	remaining: 47.6s
494:	learn: 0.9082021	total: 46.5s	remaining: 47.5s
495:	learn: 0.9075118	total: 46.6s	remaining: 47.4s
496:	learn: 0.9056834	total: 46.7s	remaining: 47.3s
497:	learn: 0.9038373	

637:	learn: 0.7305557	total: 1m	remaining: 34s
638:	learn: 0.7294954	total: 1m	remaining: 34s
639:	learn: 0.7285727	total: 1m	remaining: 33.9s
640:	learn: 0.7273634	total: 1m	remaining: 33.8s
641:	learn: 0.7263383	total: 1m	remaining: 33.7s
642:	learn: 0.7249042	total: 1m	remaining: 33.6s
643:	learn: 0.7237483	total: 1m	remaining: 33.5s
644:	learn: 0.7226301	total: 1m	remaining: 33.4s
645:	learn: 0.7212141	total: 1m	remaining: 33.3s
646:	learn: 0.7209484	total: 1m	remaining: 33.2s
647:	learn: 0.7205070	total: 1m	remaining: 33.1s
648:	learn: 0.7189825	total: 1m 1s	remaining: 33s
649:	learn: 0.7181476	total: 1m 1s	remaining: 32.9s
650:	learn: 0.7174124	total: 1m 1s	remaining: 32.8s
651:	learn: 0.7168064	total: 1m 1s	remaining: 32.7s
652:	learn: 0.7165921	total: 1m 1s	remaining: 32.6s
653:	learn: 0.7163804	total: 1m 1s	remaining: 32.5s
654:	learn: 0.7148776	total: 1m 1s	remaining: 32.5s
655:	learn: 0.7136712	total: 1m 1s	remaining: 32.4s
656:	learn: 0.7123188	total: 1m 1s	remaining: 32.3s

...

285:	learn: 1.2047230	total: 26.9s	remaining: 1m 7s
286:	learn: 1.2017640	total: 27s	remaining: 1m 6s
287:	learn: 1.2003117	total: 27.1s	remaining: 1m 6s
288:	learn: 1.1971445	total: 27.1s	remaining: 1m 6s
289:	learn: 1.1957870	total: 27.2s	remaining: 1m 6s
290:	learn: 1.1928567	total: 27.3s	remaining: 1m 6s
291:	learn: 1.1903046	total: 27.4s	remaining: 1m 6s
292:	learn: 1.1875533	total: 27.5s	remaining: 1m 6s
293:	learn: 1.1848118	total: 27.6s	remaining: 1m 6s
294:	learn: 1.1804889	total: 27.7s	remaining: 1m 6s
295:	learn: 1.1798447	total: 27.8s	remaining: 1m 6s
296:	learn: 1.1766243	total: 27.9s	remaining: 1m 6s
297:	learn: 1.1729381	total: 28s	remaining: 1m 5s
298:	learn: 1.1685215	total: 28.1s	remaining: 1m 5s
299:	learn: 1.1657193	total: 28.2s	remaining: 1m 5s
300:	learn: 1.1646864	total: 28.3s	remaining: 1m 5s
301:	learn: 1.1623152	total: 28.4s	remaining: 1m 5s
302:	learn: 1.1613545	total: 28.5s	remaining: 1m 5s
303:	learn: 1.1580698	total: 28.6s	remaining: 1m 5s
304:	learn: 1.15

447:	learn: 0.9119034	total: 42s	remaining: 51.8s
448:	learn: 0.9112853	total: 42.1s	remaining: 51.7s
449:	learn: 0.9106316	total: 42.2s	remaining: 51.6s
450:	learn: 0.9089785	total: 42.3s	remaining: 51.5s
451:	learn: 0.9073724	total: 42.4s	remaining: 51.4s
452:	learn: 0.9052688	total: 42.5s	remaining: 51.3s
453:	learn: 0.9033779	total: 42.6s	remaining: 51.2s
454:	learn: 0.9011773	total: 42.7s	remaining: 51.1s
455:	learn: 0.8986783	total: 42.8s	remaining: 51s
456:	learn: 0.8964388	total: 42.9s	remaining: 51s
457:	learn: 0.8945634	total: 43s	remaining: 50.9s
458:	learn: 0.8940123	total: 43.1s	remaining: 50.8s
459:	learn: 0.8907594	total: 43.2s	remaining: 50.7s
460:	learn: 0.8892475	total: 43.3s	remaining: 50.6s
461:	learn: 0.8872025	total: 43.4s	remaining: 50.5s
462:	learn: 0.8846214	total: 43.5s	remaining: 50.4s
463:	learn: 0.8832860	total: 43.5s	remaining: 50.3s
464:	learn: 0.8812156	total: 43.6s	remaining: 50.2s
465:	learn: 0.8804507	total: 43.7s	remaining: 50.1s
466:	learn: 0.879710

606:	learn: 0.6994203	total: 57s	remaining: 36.9s
607:	learn: 0.6990342	total: 57.1s	remaining: 36.8s
608:	learn: 0.6981220	total: 57.2s	remaining: 36.7s
609:	learn: 0.6970365	total: 57.3s	remaining: 36.6s
610:	learn: 0.6958638	total: 57.4s	remaining: 36.5s
611:	learn: 0.6944467	total: 57.5s	remaining: 36.4s
612:	learn: 0.6928487	total: 57.6s	remaining: 36.3s
613:	learn: 0.6915409	total: 57.7s	remaining: 36.2s
614:	learn: 0.6897492	total: 57.7s	remaining: 36.1s
615:	learn: 0.6883275	total: 57.8s	remaining: 36.1s
616:	learn: 0.6874874	total: 57.9s	remaining: 36s
617:	learn: 0.6859739	total: 58s	remaining: 35.9s
618:	learn: 0.6850612	total: 58.1s	remaining: 35.8s
619:	learn: 0.6840514	total: 58.2s	remaining: 35.7s
620:	learn: 0.6835728	total: 58.3s	remaining: 35.6s
621:	learn: 0.6825979	total: 58.4s	remaining: 35.5s
622:	learn: 0.6814016	total: 58.5s	remaining: 35.4s
623:	learn: 0.6802608	total: 58.6s	remaining: 35.3s
624:	learn: 0.6793794	total: 58.7s	remaining: 35.2s
625:	learn: 0.6790

765:	learn: 0.5529656	total: 1m 11s	remaining: 22s
766:	learn: 0.5520508	total: 1m 12s	remaining: 21.9s
767:	learn: 0.5514877	total: 1m 12s	remaining: 21.8s
768:	learn: 0.5505307	total: 1m 12s	remaining: 21.7s
769:	learn: 0.5498396	total: 1m 12s	remaining: 21.6s
770:	learn: 0.5486380	total: 1m 12s	remaining: 21.5s
771:	learn: 0.5479340	total: 1m 12s	remaining: 21.4s
772:	learn: 0.5475798	total: 1m 12s	remaining: 21.3s
773:	learn: 0.5466111	total: 1m 12s	remaining: 21.2s
774:	learn: 0.5456202	total: 1m 12s	remaining: 21.1s
775:	learn: 0.5443576	total: 1m 12s	remaining: 21s
776:	learn: 0.5436925	total: 1m 12s	remaining: 20.9s
777:	learn: 0.5435511	total: 1m 13s	remaining: 20.9s
778:	learn: 0.5425079	total: 1m 13s	remaining: 20.8s
779:	learn: 0.5418507	total: 1m 13s	remaining: 20.7s
780:	learn: 0.5410674	total: 1m 13s	remaining: 20.6s
781:	learn: 0.5405885	total: 1m 13s	remaining: 20.5s
782:	learn: 0.5403879	total: 1m 13s	remaining: 20.4s
783:	learn: 0.5392877	total: 1m 13s	remaining: 20.

923:	learn: 0.4553201	total: 1m 26s	remaining: 7.14s
924:	learn: 0.4547159	total: 1m 26s	remaining: 7.05s
925:	learn: 0.4545334	total: 1m 27s	remaining: 6.95s
926:	learn: 0.4540758	total: 1m 27s	remaining: 6.86s
927:	learn: 0.4532486	total: 1m 27s	remaining: 6.77s
928:	learn: 0.4526253	total: 1m 27s	remaining: 6.67s
929:	learn: 0.4520863	total: 1m 27s	remaining: 6.58s
930:	learn: 0.4516539	total: 1m 27s	remaining: 6.48s
931:	learn: 0.4509097	total: 1m 27s	remaining: 6.39s
932:	learn: 0.4503347	total: 1m 27s	remaining: 6.3s
933:	learn: 0.4500481	total: 1m 27s	remaining: 6.2s
934:	learn: 0.4498029	total: 1m 27s	remaining: 6.11s
935:	learn: 0.4492649	total: 1m 27s	remaining: 6.01s
936:	learn: 0.4486293	total: 1m 28s	remaining: 5.92s
937:	learn: 0.4478377	total: 1m 28s	remaining: 5.83s
938:	learn: 0.4472587	total: 1m 28s	remaining: 5.73s
939:	learn: 0.4471575	total: 1m 28s	remaining: 5.64s
940:	learn: 0.4463319	total: 1m 28s	remaining: 5.54s
941:	learn: 0.4454281	total: 1m 28s	remaining: 5

In [8]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.920865777517682
0.9332507196646289
0.9096273087106167
0.9417040749673701
0.8939747430210728
0.925772041224721
0.5817834958508027
0.5540886492279933
0.5689455597256184
0.6079699518516214
0.5765921465944783
0.6013211718543022


In [9]:
train_size = 4000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.050969
0:	learn: 3.2152934	total: 134ms	remaining: 2m 13s
1:	learn: 3.1653497	total: 240ms	remaining: 1m 59s
2:	learn: 3.1183236	total: 347ms	remaining: 1m 55s
3:	learn: 3.0742925	total: 453ms	remaining: 1m 52s
4:	learn: 3.0310285	total: 559ms	remaining: 1m 51s
5:	learn: 2.9922169	total: 665ms	remaining: 1m 50s
6:	learn: 2.9572793	total: 773ms	remaining: 1m 49s
7:	learn: 2.9173323	total: 879ms	remaining: 1m 48s
8:	learn: 2.8873338	total: 985ms	remaining: 1m 48s
9:	learn: 2.8555844	total: 1.09s	remaining: 1m 47s
10:	learn: 2.8279470	total: 1.2s	remaining: 1m 47s
11:	learn: 2.8020343	total: 1.3s	remaining: 1m 47s
12:	learn: 2.7723958	total: 1.41s	remaining: 1m 47s
13:	learn: 2.7415597	total: 1.52s	remaining: 1m 46s
14:	learn: 2.7130210	total: 1.62s	remaining: 1m 46s
15:	learn: 2.6884047	total: 1.73s	remaining: 1m 46s
16:	learn: 2.6604535	total: 1.83s	remaining: 1m 46s
17:	learn: 2.6330200	total: 1.94s	remaining: 1m 45s
18:	learn: 2.6129566	total: 2.05s	remaining: 1

157:	learn: 1.7433813	total: 16.7s	remaining: 1m 29s
158:	learn: 1.7358632	total: 16.8s	remaining: 1m 28s
159:	learn: 1.7342203	total: 16.9s	remaining: 1m 28s
160:	learn: 1.7281551	total: 17s	remaining: 1m 28s
161:	learn: 1.7255567	total: 17.1s	remaining: 1m 28s
162:	learn: 1.7232938	total: 17.2s	remaining: 1m 28s
163:	learn: 1.7213119	total: 17.3s	remaining: 1m 28s
164:	learn: 1.7126958	total: 17.4s	remaining: 1m 28s
165:	learn: 1.7113937	total: 17.5s	remaining: 1m 28s
166:	learn: 1.7101684	total: 17.7s	remaining: 1m 28s
167:	learn: 1.7060643	total: 17.8s	remaining: 1m 27s
168:	learn: 1.7017586	total: 17.9s	remaining: 1m 27s
169:	learn: 1.7000747	total: 18s	remaining: 1m 27s
170:	learn: 1.6956748	total: 18.1s	remaining: 1m 27s
171:	learn: 1.6909474	total: 18.2s	remaining: 1m 27s
172:	learn: 1.6893687	total: 18.3s	remaining: 1m 27s
173:	learn: 1.6877276	total: 18.4s	remaining: 1m 27s
174:	learn: 1.6846563	total: 18.5s	remaining: 1m 27s
175:	learn: 1.6792826	total: 18.6s	remaining: 1m 2

313:	learn: 1.2840469	total: 33.2s	remaining: 1m 12s
314:	learn: 1.2805460	total: 33.3s	remaining: 1m 12s
315:	learn: 1.2757817	total: 33.4s	remaining: 1m 12s
316:	learn: 1.2751362	total: 33.5s	remaining: 1m 12s
317:	learn: 1.2720556	total: 33.6s	remaining: 1m 12s
318:	learn: 1.2697203	total: 33.7s	remaining: 1m 11s
319:	learn: 1.2676286	total: 33.8s	remaining: 1m 11s
320:	learn: 1.2649440	total: 33.9s	remaining: 1m 11s
321:	learn: 1.2635167	total: 34s	remaining: 1m 11s
322:	learn: 1.2603630	total: 34.1s	remaining: 1m 11s
323:	learn: 1.2583154	total: 34.2s	remaining: 1m 11s
324:	learn: 1.2577350	total: 34.3s	remaining: 1m 11s
325:	learn: 1.2541817	total: 34.4s	remaining: 1m 11s
326:	learn: 1.2509961	total: 34.5s	remaining: 1m 11s
327:	learn: 1.2484515	total: 34.7s	remaining: 1m 10s
328:	learn: 1.2478357	total: 34.8s	remaining: 1m 10s
329:	learn: 1.2443047	total: 34.9s	remaining: 1m 10s
330:	learn: 1.2437138	total: 35s	remaining: 1m 10s
331:	learn: 1.2402740	total: 35.1s	remaining: 1m 1

473:	learn: 0.9941903	total: 50.1s	remaining: 55.6s
474:	learn: 0.9934261	total: 50.2s	remaining: 55.5s
475:	learn: 0.9913603	total: 50.3s	remaining: 55.4s
476:	learn: 0.9899803	total: 50.4s	remaining: 55.3s
477:	learn: 0.9880833	total: 50.5s	remaining: 55.2s
478:	learn: 0.9875295	total: 50.6s	remaining: 55.1s
479:	learn: 0.9860748	total: 50.7s	remaining: 54.9s
480:	learn: 0.9839128	total: 50.8s	remaining: 54.8s
481:	learn: 0.9824317	total: 50.9s	remaining: 54.7s
482:	learn: 0.9819202	total: 51s	remaining: 54.6s
483:	learn: 0.9807336	total: 51.1s	remaining: 54.5s
484:	learn: 0.9803776	total: 51.2s	remaining: 54.4s
485:	learn: 0.9796540	total: 51.4s	remaining: 54.3s
486:	learn: 0.9779665	total: 51.5s	remaining: 54.2s
487:	learn: 0.9776222	total: 51.6s	remaining: 54.1s
488:	learn: 0.9772866	total: 51.7s	remaining: 54s
489:	learn: 0.9752530	total: 51.8s	remaining: 53.9s
490:	learn: 0.9749268	total: 51.9s	remaining: 53.8s
491:	learn: 0.9746108	total: 52s	remaining: 53.7s
492:	learn: 0.9742

633:	learn: 0.8061183	total: 1m 7s	remaining: 38.7s
634:	learn: 0.8044923	total: 1m 7s	remaining: 38.6s
635:	learn: 0.8034382	total: 1m 7s	remaining: 38.5s
636:	learn: 0.8020867	total: 1m 7s	remaining: 38.4s
637:	learn: 0.8007600	total: 1m 7s	remaining: 38.3s
638:	learn: 0.7989680	total: 1m 7s	remaining: 38.2s
639:	learn: 0.7972135	total: 1m 7s	remaining: 38.1s
640:	learn: 0.7968428	total: 1m 7s	remaining: 37.9s
641:	learn: 0.7954666	total: 1m 7s	remaining: 37.8s
642:	learn: 0.7952567	total: 1m 7s	remaining: 37.7s
643:	learn: 0.7940095	total: 1m 8s	remaining: 37.6s
644:	learn: 0.7932789	total: 1m 8s	remaining: 37.5s
645:	learn: 0.7927260	total: 1m 8s	remaining: 37.4s
646:	learn: 0.7916052	total: 1m 8s	remaining: 37.3s
647:	learn: 0.7912668	total: 1m 8s	remaining: 37.2s
648:	learn: 0.7909378	total: 1m 8s	remaining: 37.1s
649:	learn: 0.7895230	total: 1m 8s	remaining: 37s
650:	learn: 0.7886590	total: 1m 8s	remaining: 36.9s
651:	learn: 0.7866594	total: 1m 8s	remaining: 36.8s
652:	learn: 0.

...

253:	learn: 1.3067958	total: 26.8s	remaining: 1m 18s
254:	learn: 1.3042755	total: 26.9s	remaining: 1m 18s
255:	learn: 1.3005450	total: 27s	remaining: 1m 18s
256:	learn: 1.2963778	total: 27.1s	remaining: 1m 18s
257:	learn: 1.2934335	total: 27.2s	remaining: 1m 18s
258:	learn: 1.2899576	total: 27.3s	remaining: 1m 18s
259:	learn: 1.2885759	total: 27.4s	remaining: 1m 18s
260:	learn: 1.2875498	total: 27.5s	remaining: 1m 17s
261:	learn: 1.2845221	total: 27.6s	remaining: 1m 17s
262:	learn: 1.2813485	total: 27.7s	remaining: 1m 17s
263:	learn: 1.2767657	total: 27.8s	remaining: 1m 17s
264:	learn: 1.2758508	total: 27.9s	remaining: 1m 17s
265:	learn: 1.2720519	total: 28.1s	remaining: 1m 17s
266:	learn: 1.2714688	total: 28.2s	remaining: 1m 17s
267:	learn: 1.2677930	total: 28.3s	remaining: 1m 17s
268:	learn: 1.2669788	total: 28.4s	remaining: 1m 17s
269:	learn: 1.2631577	total: 28.5s	remaining: 1m 16s
270:	learn: 1.2606142	total: 28.6s	remaining: 1m 16s
271:	learn: 1.2571992	total: 28.7s	remaining: 1m

411:	learn: 1.0195898	total: 43.4s	remaining: 1m 2s
412:	learn: 1.0173175	total: 43.6s	remaining: 1m 1s
413:	learn: 1.0156928	total: 43.7s	remaining: 1m 1s
414:	learn: 1.0153624	total: 43.8s	remaining: 1m 1s
415:	learn: 1.0135949	total: 43.9s	remaining: 1m 1s
416:	learn: 1.0108549	total: 44s	remaining: 1m 1s
417:	learn: 1.0090537	total: 44.1s	remaining: 1m 1s
418:	learn: 1.0069636	total: 44.2s	remaining: 1m 1s
419:	learn: 1.0049456	total: 44.3s	remaining: 1m 1s
420:	learn: 1.0045429	total: 44.4s	remaining: 1m 1s
421:	learn: 1.0025731	total: 44.5s	remaining: 1m
422:	learn: 1.0019812	total: 44.6s	remaining: 1m
423:	learn: 1.0007994	total: 44.7s	remaining: 1m
424:	learn: 0.9986977	total: 44.8s	remaining: 1m
425:	learn: 0.9978989	total: 44.9s	remaining: 1m
426:	learn: 0.9960870	total: 45s	remaining: 1m
427:	learn: 0.9948238	total: 45.2s	remaining: 1m
428:	learn: 0.9940502	total: 45.3s	remaining: 1m
429:	learn: 0.9917425	total: 45.4s	remaining: 1m
430:	learn: 0.9910704	total: 45.5s	remainin

571:	learn: 0.8268542	total: 1m	remaining: 45.1s
572:	learn: 0.8266463	total: 1m	remaining: 45s
573:	learn: 0.8254202	total: 1m	remaining: 44.9s
574:	learn: 0.8237751	total: 1m	remaining: 44.8s
575:	learn: 0.8225254	total: 1m	remaining: 44.7s
576:	learn: 0.8207245	total: 1m	remaining: 44.6s
577:	learn: 0.8199191	total: 1m	remaining: 44.5s
578:	learn: 0.8197050	total: 1m 1s	remaining: 44.4s
579:	learn: 0.8189624	total: 1m 1s	remaining: 44.3s
580:	learn: 0.8174001	total: 1m 1s	remaining: 44.2s
581:	learn: 0.8164495	total: 1m 1s	remaining: 44.1s
582:	learn: 0.8148536	total: 1m 1s	remaining: 44s
583:	learn: 0.8130303	total: 1m 1s	remaining: 43.9s
584:	learn: 0.8123917	total: 1m 1s	remaining: 43.8s
585:	learn: 0.8116208	total: 1m 1s	remaining: 43.7s
586:	learn: 0.8104415	total: 1m 1s	remaining: 43.6s
587:	learn: 0.8093697	total: 1m 2s	remaining: 43.5s
588:	learn: 0.8078812	total: 1m 2s	remaining: 43.4s
589:	learn: 0.8064541	total: 1m 2s	remaining: 43.3s
590:	learn: 0.8059875	total: 1m 2s	re

729:	learn: 0.6867650	total: 1m 17s	remaining: 28.5s
730:	learn: 0.6854088	total: 1m 17s	remaining: 28.4s
731:	learn: 0.6852350	total: 1m 17s	remaining: 28.3s
732:	learn: 0.6844321	total: 1m 17s	remaining: 28.2s
733:	learn: 0.6836549	total: 1m 17s	remaining: 28.1s
734:	learn: 0.6828825	total: 1m 17s	remaining: 28s
735:	learn: 0.6819489	total: 1m 17s	remaining: 27.9s
736:	learn: 0.6809825	total: 1m 17s	remaining: 27.8s
737:	learn: 0.6800825	total: 1m 17s	remaining: 27.6s
738:	learn: 0.6789978	total: 1m 17s	remaining: 27.5s
739:	learn: 0.6787348	total: 1m 18s	remaining: 27.4s
740:	learn: 0.6779822	total: 1m 18s	remaining: 27.3s
741:	learn: 0.6771484	total: 1m 18s	remaining: 27.2s
742:	learn: 0.6758766	total: 1m 18s	remaining: 27.1s
743:	learn: 0.6755202	total: 1m 18s	remaining: 27s
744:	learn: 0.6743213	total: 1m 18s	remaining: 26.9s
745:	learn: 0.6729238	total: 1m 18s	remaining: 26.8s
746:	learn: 0.6713845	total: 1m 18s	remaining: 26.7s
747:	learn: 0.6712305	total: 1m 18s	remaining: 26.

885:	learn: 0.5765860	total: 1m 33s	remaining: 12s
886:	learn: 0.5759659	total: 1m 33s	remaining: 11.9s
887:	learn: 0.5748549	total: 1m 33s	remaining: 11.8s
888:	learn: 0.5743188	total: 1m 33s	remaining: 11.7s
889:	learn: 0.5738305	total: 1m 33s	remaining: 11.6s
890:	learn: 0.5729250	total: 1m 34s	remaining: 11.5s
891:	learn: 0.5728102	total: 1m 34s	remaining: 11.4s
892:	learn: 0.5721849	total: 1m 34s	remaining: 11.3s
893:	learn: 0.5712333	total: 1m 34s	remaining: 11.2s
894:	learn: 0.5701704	total: 1m 34s	remaining: 11.1s
895:	learn: 0.5700419	total: 1m 34s	remaining: 11s
896:	learn: 0.5697619	total: 1m 34s	remaining: 10.9s
897:	learn: 0.5689363	total: 1m 34s	remaining: 10.8s
898:	learn: 0.5681158	total: 1m 34s	remaining: 10.7s
899:	learn: 0.5676968	total: 1m 34s	remaining: 10.6s
900:	learn: 0.5670922	total: 1m 35s	remaining: 10.4s
901:	learn: 0.5662780	total: 1m 35s	remaining: 10.3s
902:	learn: 0.5658985	total: 1m 35s	remaining: 10.2s
903:	learn: 0.5651292	total: 1m 35s	remaining: 10.

In [10]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.9070811485341782
0.8952226324010796
0.9128258063051772
0.9401118998753776
0.8693866638774517
0.9178587402118052
0.5942965869275221
0.6176563182238676
0.5410747716140707
0.5895644611062716
0.613905730524666
0.6092816531687344


In [11]:
train_size = 5000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.052798
0:	learn: 3.1427672	total: 139ms	remaining: 2m 19s
1:	learn: 3.0893083	total: 255ms	remaining: 2m 7s
2:	learn: 3.0423669	total: 371ms	remaining: 2m 3s
3:	learn: 2.9941208	total: 487ms	remaining: 2m 1s
4:	learn: 2.9502033	total: 603ms	remaining: 1m 59s
5:	learn: 2.9087042	total: 718ms	remaining: 1m 58s
6:	learn: 2.8671645	total: 834ms	remaining: 1m 58s
7:	learn: 2.8273798	total: 950ms	remaining: 1m 57s
8:	learn: 2.7904605	total: 1.07s	remaining: 1m 57s
9:	learn: 2.7534496	total: 1.18s	remaining: 1m 57s
10:	learn: 2.7235721	total: 1.3s	remaining: 1m 56s
11:	learn: 2.6913782	total: 1.41s	remaining: 1m 56s
12:	learn: 2.6603072	total: 1.53s	remaining: 1m 56s
13:	learn: 2.6326268	total: 1.65s	remaining: 1m 56s
14:	learn: 2.6084208	total: 1.76s	remaining: 1m 55s
15:	learn: 2.5833780	total: 1.88s	remaining: 1m 55s
16:	learn: 2.5603006	total: 1.99s	remaining: 1m 55s
17:	learn: 2.5394693	total: 2.11s	remaining: 1m 55s
18:	learn: 2.5161762	total: 2.23s	remaining: 1m 

157:	learn: 1.6949181	total: 18.2s	remaining: 1m 37s
158:	learn: 1.6932188	total: 18.3s	remaining: 1m 36s
159:	learn: 1.6916838	total: 18.4s	remaining: 1m 36s
160:	learn: 1.6905903	total: 18.6s	remaining: 1m 36s
161:	learn: 1.6877500	total: 18.7s	remaining: 1m 36s
162:	learn: 1.6862906	total: 18.8s	remaining: 1m 36s
163:	learn: 1.6830309	total: 18.9s	remaining: 1m 36s
164:	learn: 1.6783727	total: 19s	remaining: 1m 36s
165:	learn: 1.6763304	total: 19.1s	remaining: 1m 36s
166:	learn: 1.6743119	total: 19.3s	remaining: 1m 36s
167:	learn: 1.6703956	total: 19.4s	remaining: 1m 35s
168:	learn: 1.6634353	total: 19.5s	remaining: 1m 35s
169:	learn: 1.6596454	total: 19.6s	remaining: 1m 35s
170:	learn: 1.6575635	total: 19.7s	remaining: 1m 35s
171:	learn: 1.6526531	total: 19.8s	remaining: 1m 35s
172:	learn: 1.6507451	total: 19.9s	remaining: 1m 35s
173:	learn: 1.6495574	total: 20.1s	remaining: 1m 35s
174:	learn: 1.6459083	total: 20.2s	remaining: 1m 35s
175:	learn: 1.6444558	total: 20.3s	remaining: 1m

313:	learn: 1.2877899	total: 36.1s	remaining: 1m 18s
314:	learn: 1.2866044	total: 36.3s	remaining: 1m 18s
315:	learn: 1.2860693	total: 36.4s	remaining: 1m 18s
316:	learn: 1.2849275	total: 36.5s	remaining: 1m 18s
317:	learn: 1.2822179	total: 36.6s	remaining: 1m 18s
318:	learn: 1.2811677	total: 36.7s	remaining: 1m 18s
319:	learn: 1.2802001	total: 36.8s	remaining: 1m 18s
320:	learn: 1.2794506	total: 36.9s	remaining: 1m 18s
321:	learn: 1.2778797	total: 37.1s	remaining: 1m 18s
322:	learn: 1.2751535	total: 37.2s	remaining: 1m 17s
323:	learn: 1.2722233	total: 37.3s	remaining: 1m 17s
324:	learn: 1.2685679	total: 37.4s	remaining: 1m 17s
325:	learn: 1.2679179	total: 37.5s	remaining: 1m 17s
326:	learn: 1.2655212	total: 37.6s	remaining: 1m 17s
327:	learn: 1.2650774	total: 37.7s	remaining: 1m 17s
328:	learn: 1.2640801	total: 37.9s	remaining: 1m 17s
329:	learn: 1.2625438	total: 38s	remaining: 1m 17s
330:	learn: 1.2584936	total: 38.1s	remaining: 1m 16s
331:	learn: 1.2576820	total: 38.2s	remaining: 1m

471:	learn: 1.0201330	total: 54.4s	remaining: 1m
472:	learn: 1.0194875	total: 54.5s	remaining: 1m
473:	learn: 1.0173564	total: 54.6s	remaining: 1m
474:	learn: 1.0155949	total: 54.7s	remaining: 1m
475:	learn: 1.0145724	total: 54.8s	remaining: 1m
476:	learn: 1.0138604	total: 54.9s	remaining: 1m
477:	learn: 1.0120382	total: 55s	remaining: 1m
478:	learn: 1.0101988	total: 55.2s	remaining: 60s
479:	learn: 1.0097238	total: 55.3s	remaining: 59.9s
480:	learn: 1.0092909	total: 55.4s	remaining: 59.8s
481:	learn: 1.0079236	total: 55.5s	remaining: 59.6s
482:	learn: 1.0076531	total: 55.6s	remaining: 59.5s
483:	learn: 1.0062971	total: 55.7s	remaining: 59.4s
484:	learn: 1.0048368	total: 55.8s	remaining: 59.3s
485:	learn: 1.0033721	total: 56s	remaining: 59.2s
486:	learn: 1.0018071	total: 56.1s	remaining: 59.1s
487:	learn: 1.0004779	total: 56.2s	remaining: 59s
488:	learn: 1.0000147	total: 56.3s	remaining: 58.8s
489:	learn: 0.9983050	total: 56.4s	remaining: 58.7s
490:	learn: 0.9973062	total: 56.5s	remain

631:	learn: 0.8448669	total: 1m 12s	remaining: 42.4s
632:	learn: 0.8436657	total: 1m 12s	remaining: 42.3s
633:	learn: 0.8424910	total: 1m 13s	remaining: 42.1s
634:	learn: 0.8418644	total: 1m 13s	remaining: 42s
635:	learn: 0.8408318	total: 1m 13s	remaining: 41.9s
636:	learn: 0.8389723	total: 1m 13s	remaining: 41.8s
637:	learn: 0.8380883	total: 1m 13s	remaining: 41.7s
638:	learn: 0.8366716	total: 1m 13s	remaining: 41.6s
639:	learn: 0.8364762	total: 1m 13s	remaining: 41.5s
640:	learn: 0.8358405	total: 1m 13s	remaining: 41.3s
641:	learn: 0.8349635	total: 1m 13s	remaining: 41.2s
642:	learn: 0.8335874	total: 1m 14s	remaining: 41.1s
643:	learn: 0.8328094	total: 1m 14s	remaining: 41s
644:	learn: 0.8326281	total: 1m 14s	remaining: 40.9s
645:	learn: 0.8318585	total: 1m 14s	remaining: 40.8s
646:	learn: 0.8311522	total: 1m 14s	remaining: 40.6s
647:	learn: 0.8303289	total: 1m 14s	remaining: 40.5s
648:	learn: 0.8290849	total: 1m 14s	remaining: 40.4s
649:	learn: 0.8282950	total: 1m 14s	remaining: 40.

...

243:	learn: 1.4546934	total: 27.9s	remaining: 1m 26s
244:	learn: 1.4522823	total: 28s	remaining: 1m 26s
245:	learn: 1.4509249	total: 28.1s	remaining: 1m 26s
246:	learn: 1.4501308	total: 28.2s	remaining: 1m 26s
247:	learn: 1.4469436	total: 28.4s	remaining: 1m 25s
248:	learn: 1.4438885	total: 28.5s	remaining: 1m 25s
249:	learn: 1.4413196	total: 28.6s	remaining: 1m 25s
250:	learn: 1.4369563	total: 28.7s	remaining: 1m 25s
251:	learn: 1.4361982	total: 28.8s	remaining: 1m 25s
252:	learn: 1.4329240	total: 28.9s	remaining: 1m 25s
253:	learn: 1.4297804	total: 29s	remaining: 1m 25s
254:	learn: 1.4259587	total: 29.2s	remaining: 1m 25s
255:	learn: 1.4252158	total: 29.3s	remaining: 1m 25s
256:	learn: 1.4228305	total: 29.4s	remaining: 1m 24s
257:	learn: 1.4204129	total: 29.5s	remaining: 1m 24s
258:	learn: 1.4194441	total: 29.6s	remaining: 1m 24s
259:	learn: 1.4177294	total: 29.7s	remaining: 1m 24s
260:	learn: 1.4146432	total: 29.8s	remaining: 1m 24s
261:	learn: 1.4091766	total: 30s	remaining: 1m 24s

399:	learn: 1.1399558	total: 45.8s	remaining: 1m 8s
400:	learn: 1.1390654	total: 45.9s	remaining: 1m 8s
401:	learn: 1.1381460	total: 46s	remaining: 1m 8s
402:	learn: 1.1346074	total: 46.1s	remaining: 1m 8s
403:	learn: 1.1316807	total: 46.2s	remaining: 1m 8s
404:	learn: 1.1303237	total: 46.4s	remaining: 1m 8s
405:	learn: 1.1278573	total: 46.5s	remaining: 1m 7s
406:	learn: 1.1274992	total: 46.6s	remaining: 1m 7s
407:	learn: 1.1258851	total: 46.7s	remaining: 1m 7s
408:	learn: 1.1255154	total: 46.8s	remaining: 1m 7s
409:	learn: 1.1240751	total: 46.9s	remaining: 1m 7s
410:	learn: 1.1220634	total: 47s	remaining: 1m 7s
411:	learn: 1.1200040	total: 47.2s	remaining: 1m 7s
412:	learn: 1.1175909	total: 47.3s	remaining: 1m 7s
413:	learn: 1.1157913	total: 47.4s	remaining: 1m 7s
414:	learn: 1.1150314	total: 47.5s	remaining: 1m 6s
415:	learn: 1.1136800	total: 47.6s	remaining: 1m 6s
416:	learn: 1.1132268	total: 47.7s	remaining: 1m 6s
417:	learn: 1.1106815	total: 47.8s	remaining: 1m 6s
418:	learn: 1.10

559:	learn: 0.9263047	total: 1m 4s	remaining: 50.4s
560:	learn: 0.9260314	total: 1m 4s	remaining: 50.3s
561:	learn: 0.9247343	total: 1m 4s	remaining: 50.1s
562:	learn: 0.9244004	total: 1m 4s	remaining: 50s
563:	learn: 0.9228828	total: 1m 4s	remaining: 49.9s
564:	learn: 0.9226288	total: 1m 4s	remaining: 49.8s
565:	learn: 0.9211466	total: 1m 4s	remaining: 49.7s
566:	learn: 0.9208070	total: 1m 4s	remaining: 49.6s
567:	learn: 0.9197880	total: 1m 5s	remaining: 49.5s
568:	learn: 0.9187828	total: 1m 5s	remaining: 49.3s
569:	learn: 0.9179545	total: 1m 5s	remaining: 49.2s
570:	learn: 0.9176970	total: 1m 5s	remaining: 49.1s
571:	learn: 0.9162016	total: 1m 5s	remaining: 49s
572:	learn: 0.9149734	total: 1m 5s	remaining: 48.9s
573:	learn: 0.9144567	total: 1m 5s	remaining: 48.8s
574:	learn: 0.9133264	total: 1m 5s	remaining: 48.7s
575:	learn: 0.9123672	total: 1m 5s	remaining: 48.5s
576:	learn: 0.9112539	total: 1m 6s	remaining: 48.4s
577:	learn: 0.9101013	total: 1m 6s	remaining: 48.3s
578:	learn: 0.90

717:	learn: 0.7928123	total: 1m 22s	remaining: 32.3s
718:	learn: 0.7915871	total: 1m 22s	remaining: 32.2s
719:	learn: 0.7901266	total: 1m 22s	remaining: 32.1s
720:	learn: 0.7891638	total: 1m 22s	remaining: 31.9s
721:	learn: 0.7880789	total: 1m 22s	remaining: 31.8s
722:	learn: 0.7864053	total: 1m 22s	remaining: 31.7s
723:	learn: 0.7862025	total: 1m 22s	remaining: 31.6s
724:	learn: 0.7849462	total: 1m 23s	remaining: 31.5s
725:	learn: 0.7844623	total: 1m 23s	remaining: 31.4s
726:	learn: 0.7842769	total: 1m 23s	remaining: 31.3s
727:	learn: 0.7840790	total: 1m 23s	remaining: 31.1s
728:	learn: 0.7830081	total: 1m 23s	remaining: 31s
729:	learn: 0.7820318	total: 1m 23s	remaining: 30.9s
730:	learn: 0.7811991	total: 1m 23s	remaining: 30.8s
731:	learn: 0.7793558	total: 1m 23s	remaining: 30.7s
732:	learn: 0.7782301	total: 1m 23s	remaining: 30.6s
733:	learn: 0.7775924	total: 1m 24s	remaining: 30.5s
734:	learn: 0.7762320	total: 1m 24s	remaining: 30.3s
735:	learn: 0.7755667	total: 1m 24s	remaining: 3

873:	learn: 0.6755382	total: 1m 40s	remaining: 14.4s
874:	learn: 0.6747789	total: 1m 40s	remaining: 14.3s
875:	learn: 0.6737793	total: 1m 40s	remaining: 14.2s
876:	learn: 0.6729872	total: 1m 40s	remaining: 14.1s
877:	learn: 0.6728814	total: 1m 40s	remaining: 14s
878:	learn: 0.6726721	total: 1m 40s	remaining: 13.9s
879:	learn: 0.6716516	total: 1m 40s	remaining: 13.7s
880:	learn: 0.6709228	total: 1m 40s	remaining: 13.6s
881:	learn: 0.6698673	total: 1m 41s	remaining: 13.5s
882:	learn: 0.6693601	total: 1m 41s	remaining: 13.4s
883:	learn: 0.6684738	total: 1m 41s	remaining: 13.3s
884:	learn: 0.6671632	total: 1m 41s	remaining: 13.2s
885:	learn: 0.6667549	total: 1m 41s	remaining: 13.1s
886:	learn: 0.6663075	total: 1m 41s	remaining: 12.9s
887:	learn: 0.6655827	total: 1m 41s	remaining: 12.8s
888:	learn: 0.6647408	total: 1m 41s	remaining: 12.7s
889:	learn: 0.6640100	total: 1m 41s	remaining: 12.6s
890:	learn: 0.6631339	total: 1m 42s	remaining: 12.5s
891:	learn: 0.6626547	total: 1m 42s	remaining: 1

In [12]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.8820259014393942
0.8727421455434142
0.8888988443039765
0.8899112909730238
0.872099438654101
0.8864777877224556
0.6243942336684646
0.6363522539420041
0.597192263937703
0.6498535845006439
0.6146987622829903
0.6238743036789818


In [13]:
train_size = 6000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.054341
0:	learn: 3.1645875	total: 150ms	remaining: 2m 30s
1:	learn: 3.1075045	total: 272ms	remaining: 2m 15s
2:	learn: 3.0532709	total: 394ms	remaining: 2m 10s
3:	learn: 3.0048794	total: 516ms	remaining: 2m 8s
4:	learn: 2.9572689	total: 637ms	remaining: 2m 6s
5:	learn: 2.9145110	total: 759ms	remaining: 2m 5s
6:	learn: 2.8740591	total: 881ms	remaining: 2m 4s
7:	learn: 2.8297536	total: 1s	remaining: 2m 4s
8:	learn: 2.7890781	total: 1.13s	remaining: 2m 4s
9:	learn: 2.7515478	total: 1.25s	remaining: 2m 3s
10:	learn: 2.7205263	total: 1.37s	remaining: 2m 3s
11:	learn: 2.6893746	total: 1.49s	remaining: 2m 2s
12:	learn: 2.6555200	total: 1.61s	remaining: 2m 2s
13:	learn: 2.6243221	total: 1.73s	remaining: 2m 2s
14:	learn: 2.5947376	total: 1.85s	remaining: 2m 1s
15:	learn: 2.5669149	total: 1.98s	remaining: 2m 1s
16:	learn: 2.5413701	total: 2.1s	remaining: 2m 1s
17:	learn: 2.5180598	total: 2.22s	remaining: 2m 1s
18:	learn: 2.4978770	total: 2.34s	remaining: 2m 1s
19:	learn: 2

159:	learn: 1.6928108	total: 19.4s	remaining: 1m 42s
160:	learn: 1.6896834	total: 19.6s	remaining: 1m 41s
161:	learn: 1.6883334	total: 19.7s	remaining: 1m 41s
162:	learn: 1.6850188	total: 19.8s	remaining: 1m 41s
163:	learn: 1.6840259	total: 19.9s	remaining: 1m 41s
164:	learn: 1.6811021	total: 20.1s	remaining: 1m 41s
165:	learn: 1.6794225	total: 20.2s	remaining: 1m 41s
166:	learn: 1.6761042	total: 20.3s	remaining: 1m 41s
167:	learn: 1.6744733	total: 20.4s	remaining: 1m 41s
168:	learn: 1.6735601	total: 20.5s	remaining: 1m 40s
169:	learn: 1.6681087	total: 20.7s	remaining: 1m 40s
170:	learn: 1.6632407	total: 20.8s	remaining: 1m 40s
171:	learn: 1.6618833	total: 20.9s	remaining: 1m 40s
172:	learn: 1.6571141	total: 21s	remaining: 1m 40s
173:	learn: 1.6561888	total: 21.1s	remaining: 1m 40s
174:	learn: 1.6534403	total: 21.3s	remaining: 1m 40s
175:	learn: 1.6492020	total: 21.4s	remaining: 1m 40s
176:	learn: 1.6460094	total: 21.5s	remaining: 1m 40s
177:	learn: 1.6446839	total: 21.6s	remaining: 1m

315:	learn: 1.3018536	total: 38.4s	remaining: 1m 23s
316:	learn: 1.2983927	total: 38.5s	remaining: 1m 23s
317:	learn: 1.2979162	total: 38.6s	remaining: 1m 22s
318:	learn: 1.2963981	total: 38.8s	remaining: 1m 22s
319:	learn: 1.2959534	total: 38.9s	remaining: 1m 22s
320:	learn: 1.2935957	total: 39s	remaining: 1m 22s
321:	learn: 1.2891535	total: 39.1s	remaining: 1m 22s
322:	learn: 1.2871733	total: 39.3s	remaining: 1m 22s
323:	learn: 1.2855362	total: 39.4s	remaining: 1m 22s
324:	learn: 1.2835099	total: 39.5s	remaining: 1m 22s
325:	learn: 1.2809727	total: 39.6s	remaining: 1m 21s
326:	learn: 1.2789643	total: 39.7s	remaining: 1m 21s
327:	learn: 1.2766287	total: 39.9s	remaining: 1m 21s
328:	learn: 1.2749721	total: 40s	remaining: 1m 21s
329:	learn: 1.2738809	total: 40.1s	remaining: 1m 21s
330:	learn: 1.2725814	total: 40.2s	remaining: 1m 21s
331:	learn: 1.2700659	total: 40.4s	remaining: 1m 21s
332:	learn: 1.2696285	total: 40.5s	remaining: 1m 21s
333:	learn: 1.2692091	total: 40.6s	remaining: 1m 2

473:	learn: 1.0633252	total: 57.6s	remaining: 1m 3s
474:	learn: 1.0621482	total: 57.7s	remaining: 1m 3s
475:	learn: 1.0617700	total: 57.9s	remaining: 1m 3s
476:	learn: 1.0601641	total: 58s	remaining: 1m 3s
477:	learn: 1.0591823	total: 58.1s	remaining: 1m 3s
478:	learn: 1.0589099	total: 58.2s	remaining: 1m 3s
479:	learn: 1.0566978	total: 58.3s	remaining: 1m 3s
480:	learn: 1.0552442	total: 58.5s	remaining: 1m 3s
481:	learn: 1.0534291	total: 58.6s	remaining: 1m 2s
482:	learn: 1.0513975	total: 58.7s	remaining: 1m 2s
483:	learn: 1.0503002	total: 58.8s	remaining: 1m 2s
484:	learn: 1.0500414	total: 59s	remaining: 1m 2s
485:	learn: 1.0482908	total: 59.1s	remaining: 1m 2s
486:	learn: 1.0480269	total: 59.2s	remaining: 1m 2s
487:	learn: 1.0461635	total: 59.3s	remaining: 1m 2s
488:	learn: 1.0455390	total: 59.4s	remaining: 1m 2s
489:	learn: 1.0450680	total: 59.6s	remaining: 1m 1s
490:	learn: 1.0441437	total: 59.7s	remaining: 1m 1s
491:	learn: 1.0421656	total: 59.8s	remaining: 1m 1s
492:	learn: 1.04

631:	learn: 0.8946923	total: 1m 16s	remaining: 44.7s
632:	learn: 0.8933311	total: 1m 16s	remaining: 44.6s
633:	learn: 0.8925501	total: 1m 17s	remaining: 44.5s
634:	learn: 0.8919032	total: 1m 17s	remaining: 44.4s
635:	learn: 0.8917234	total: 1m 17s	remaining: 44.3s
636:	learn: 0.8905892	total: 1m 17s	remaining: 44.1s
637:	learn: 0.8886411	total: 1m 17s	remaining: 44s
638:	learn: 0.8884650	total: 1m 17s	remaining: 43.9s
639:	learn: 0.8873243	total: 1m 17s	remaining: 43.8s
640:	learn: 0.8868207	total: 1m 17s	remaining: 43.6s
641:	learn: 0.8856347	total: 1m 18s	remaining: 43.5s
642:	learn: 0.8844323	total: 1m 18s	remaining: 43.4s
643:	learn: 0.8837963	total: 1m 18s	remaining: 43.3s
644:	learn: 0.8834767	total: 1m 18s	remaining: 43.2s
645:	learn: 0.8822526	total: 1m 18s	remaining: 43s
646:	learn: 0.8803040	total: 1m 18s	remaining: 42.9s
647:	learn: 0.8796395	total: 1m 18s	remaining: 42.8s
648:	learn: 0.8793743	total: 1m 18s	remaining: 42.7s
649:	learn: 0.8777101	total: 1m 19s	remaining: 42.

...

243:	learn: 1.4720146	total: 29.5s	remaining: 1m 31s
244:	learn: 1.4694877	total: 29.7s	remaining: 1m 31s
245:	learn: 1.4683837	total: 29.8s	remaining: 1m 31s
246:	learn: 1.4649101	total: 29.9s	remaining: 1m 31s
247:	learn: 1.4620349	total: 30s	remaining: 1m 31s
248:	learn: 1.4584788	total: 30.1s	remaining: 1m 30s
249:	learn: 1.4551440	total: 30.3s	remaining: 1m 30s
250:	learn: 1.4517465	total: 30.4s	remaining: 1m 30s
251:	learn: 1.4508394	total: 30.5s	remaining: 1m 30s
252:	learn: 1.4467514	total: 30.6s	remaining: 1m 30s
253:	learn: 1.4446239	total: 30.8s	remaining: 1m 30s
254:	learn: 1.4411191	total: 30.9s	remaining: 1m 30s
255:	learn: 1.4385292	total: 31s	remaining: 1m 30s
256:	learn: 1.4347150	total: 31.1s	remaining: 1m 29s
257:	learn: 1.4340103	total: 31.2s	remaining: 1m 29s
258:	learn: 1.4312286	total: 31.4s	remaining: 1m 29s
259:	learn: 1.4281332	total: 31.5s	remaining: 1m 29s
260:	learn: 1.4247619	total: 31.6s	remaining: 1m 29s
261:	learn: 1.4214784	total: 31.7s	remaining: 1m 2

399:	learn: 1.1593073	total: 48.4s	remaining: 1m 12s
400:	learn: 1.1580867	total: 48.5s	remaining: 1m 12s
401:	learn: 1.1567376	total: 48.7s	remaining: 1m 12s
402:	learn: 1.1548649	total: 48.8s	remaining: 1m 12s
403:	learn: 1.1530713	total: 48.9s	remaining: 1m 12s
404:	learn: 1.1516437	total: 49s	remaining: 1m 12s
405:	learn: 1.1510481	total: 49.2s	remaining: 1m 11s
406:	learn: 1.1492989	total: 49.3s	remaining: 1m 11s
407:	learn: 1.1483285	total: 49.4s	remaining: 1m 11s
408:	learn: 1.1454052	total: 49.5s	remaining: 1m 11s
409:	learn: 1.1443250	total: 49.6s	remaining: 1m 11s
410:	learn: 1.1430198	total: 49.8s	remaining: 1m 11s
411:	learn: 1.1389535	total: 49.9s	remaining: 1m 11s
412:	learn: 1.1360597	total: 50s	remaining: 1m 11s
413:	learn: 1.1349153	total: 50.1s	remaining: 1m 10s
414:	learn: 1.1330170	total: 50.3s	remaining: 1m 10s
415:	learn: 1.1324607	total: 50.4s	remaining: 1m 10s
416:	learn: 1.1314048	total: 50.5s	remaining: 1m 10s
417:	learn: 1.1302965	total: 50.6s	remaining: 1m 1

559:	learn: 0.9496373	total: 1m 7s	remaining: 53.3s
560:	learn: 0.9483112	total: 1m 7s	remaining: 53.2s
561:	learn: 0.9470313	total: 1m 8s	remaining: 53.1s
562:	learn: 0.9460941	total: 1m 8s	remaining: 52.9s
563:	learn: 0.9447015	total: 1m 8s	remaining: 52.8s
564:	learn: 0.9436067	total: 1m 8s	remaining: 52.7s
565:	learn: 0.9422777	total: 1m 8s	remaining: 52.6s
566:	learn: 0.9401399	total: 1m 8s	remaining: 52.4s
567:	learn: 0.9386655	total: 1m 8s	remaining: 52.3s
568:	learn: 0.9383692	total: 1m 8s	remaining: 52.2s
569:	learn: 0.9380359	total: 1m 9s	remaining: 52.1s
570:	learn: 0.9377042	total: 1m 9s	remaining: 52s
571:	learn: 0.9360136	total: 1m 9s	remaining: 51.8s
572:	learn: 0.9352802	total: 1m 9s	remaining: 51.7s
573:	learn: 0.9341306	total: 1m 9s	remaining: 51.6s
574:	learn: 0.9331177	total: 1m 9s	remaining: 51.5s
575:	learn: 0.9316317	total: 1m 9s	remaining: 51.4s
576:	learn: 0.9306383	total: 1m 9s	remaining: 51.2s
577:	learn: 0.9299647	total: 1m 10s	remaining: 51.1s
578:	learn: 0

715:	learn: 0.8156046	total: 1m 26s	remaining: 34.4s
716:	learn: 0.8146504	total: 1m 26s	remaining: 34.3s
717:	learn: 0.8142735	total: 1m 26s	remaining: 34.2s
718:	learn: 0.8137774	total: 1m 27s	remaining: 34s
719:	learn: 0.8125841	total: 1m 27s	remaining: 33.9s
720:	learn: 0.8116717	total: 1m 27s	remaining: 33.8s
721:	learn: 0.8110256	total: 1m 27s	remaining: 33.7s
722:	learn: 0.8099071	total: 1m 27s	remaining: 33.5s
723:	learn: 0.8096333	total: 1m 27s	remaining: 33.4s
724:	learn: 0.8087836	total: 1m 27s	remaining: 33.3s
725:	learn: 0.8073832	total: 1m 27s	remaining: 33.2s
726:	learn: 0.8071802	total: 1m 28s	remaining: 33.1s
727:	learn: 0.8058161	total: 1m 28s	remaining: 32.9s
728:	learn: 0.8055838	total: 1m 28s	remaining: 32.8s
729:	learn: 0.8046151	total: 1m 28s	remaining: 32.7s
730:	learn: 0.8044978	total: 1m 28s	remaining: 32.6s
731:	learn: 0.8039223	total: 1m 28s	remaining: 32.5s
732:	learn: 0.8027466	total: 1m 28s	remaining: 32.3s
733:	learn: 0.8020644	total: 1m 28s	remaining: 3

871:	learn: 0.7053749	total: 1m 45s	remaining: 15.5s
872:	learn: 0.7048292	total: 1m 45s	remaining: 15.4s
873:	learn: 0.7039424	total: 1m 45s	remaining: 15.3s
874:	learn: 0.7036949	total: 1m 45s	remaining: 15.1s
875:	learn: 0.7027238	total: 1m 46s	remaining: 15s
876:	learn: 0.7017231	total: 1m 46s	remaining: 14.9s
877:	learn: 0.7007267	total: 1m 46s	remaining: 14.8s
878:	learn: 0.6999375	total: 1m 46s	remaining: 14.7s
879:	learn: 0.6996245	total: 1m 46s	remaining: 14.5s
880:	learn: 0.6988199	total: 1m 46s	remaining: 14.4s
881:	learn: 0.6977569	total: 1m 46s	remaining: 14.3s
882:	learn: 0.6975519	total: 1m 46s	remaining: 14.2s
883:	learn: 0.6966586	total: 1m 47s	remaining: 14.1s
884:	learn: 0.6958531	total: 1m 47s	remaining: 13.9s
885:	learn: 0.6953172	total: 1m 47s	remaining: 13.8s
886:	learn: 0.6947629	total: 1m 47s	remaining: 13.7s
887:	learn: 0.6939238	total: 1m 47s	remaining: 13.6s
888:	learn: 0.6927587	total: 1m 47s	remaining: 13.4s
889:	learn: 0.6923067	total: 1m 47s	remaining: 1

In [14]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.8693964795993463
0.8767247856238729
0.8746257849794156
0.865108302244888
0.8526412310000807
0.8778822941484736
0.6437485144499409
0.639886985568681
0.585374424230453
0.669952003148196
0.6503034820246709
0.6732256772777039


In [15]:
train_size = 7000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.055681
0:	learn: 3.1139379	total: 167ms	remaining: 2m 46s
1:	learn: 3.0591211	total: 295ms	remaining: 2m 27s
2:	learn: 3.0045143	total: 425ms	remaining: 2m 21s
3:	learn: 2.9505710	total: 553ms	remaining: 2m 17s
4:	learn: 2.9035104	total: 683ms	remaining: 2m 15s
5:	learn: 2.8618555	total: 810ms	remaining: 2m 14s
6:	learn: 2.8174356	total: 936ms	remaining: 2m 12s
7:	learn: 2.7736332	total: 1.06s	remaining: 2m 12s
8:	learn: 2.7342665	total: 1.19s	remaining: 2m 11s
9:	learn: 2.7029095	total: 1.32s	remaining: 2m 10s
10:	learn: 2.6681984	total: 1.45s	remaining: 2m 10s
11:	learn: 2.6342818	total: 1.58s	remaining: 2m 9s
12:	learn: 2.6030280	total: 1.71s	remaining: 2m 9s
13:	learn: 2.5774571	total: 1.83s	remaining: 2m 9s
14:	learn: 2.5532027	total: 1.96s	remaining: 2m 8s
15:	learn: 2.5290574	total: 2.09s	remaining: 2m 8s
16:	learn: 2.5060299	total: 2.21s	remaining: 2m 8s
17:	learn: 2.4799516	total: 2.34s	remaining: 2m 7s
18:	learn: 2.4556342	total: 2.47s	remaining: 2m 7s


159:	learn: 1.7074119	total: 20.4s	remaining: 1m 46s
160:	learn: 1.7038746	total: 20.5s	remaining: 1m 46s
161:	learn: 1.7010368	total: 20.6s	remaining: 1m 46s
162:	learn: 1.6970268	total: 20.7s	remaining: 1m 46s
163:	learn: 1.6936817	total: 20.9s	remaining: 1m 46s
164:	learn: 1.6919445	total: 21s	remaining: 1m 46s
165:	learn: 1.6898690	total: 21.1s	remaining: 1m 46s
166:	learn: 1.6843015	total: 21.3s	remaining: 1m 46s
167:	learn: 1.6831633	total: 21.4s	remaining: 1m 45s
168:	learn: 1.6797638	total: 21.5s	remaining: 1m 45s
169:	learn: 1.6789185	total: 21.6s	remaining: 1m 45s
170:	learn: 1.6776415	total: 21.8s	remaining: 1m 45s
171:	learn: 1.6736471	total: 21.9s	remaining: 1m 45s
172:	learn: 1.6702529	total: 22s	remaining: 1m 45s
173:	learn: 1.6664766	total: 22.1s	remaining: 1m 45s
174:	learn: 1.6620499	total: 22.3s	remaining: 1m 44s
175:	learn: 1.6611153	total: 22.4s	remaining: 1m 44s
176:	learn: 1.6584738	total: 22.5s	remaining: 1m 44s
177:	learn: 1.6544138	total: 22.6s	remaining: 1m 4

315:	learn: 1.3289539	total: 40.2s	remaining: 1m 26s
316:	learn: 1.3267026	total: 40.3s	remaining: 1m 26s
317:	learn: 1.3263066	total: 40.4s	remaining: 1m 26s
318:	learn: 1.3252732	total: 40.5s	remaining: 1m 26s
319:	learn: 1.3223087	total: 40.7s	remaining: 1m 26s
320:	learn: 1.3194423	total: 40.8s	remaining: 1m 26s
321:	learn: 1.3164926	total: 40.9s	remaining: 1m 26s
322:	learn: 1.3160888	total: 41.1s	remaining: 1m 26s
323:	learn: 1.3135631	total: 41.2s	remaining: 1m 25s
324:	learn: 1.3111681	total: 41.3s	remaining: 1m 25s
325:	learn: 1.3107734	total: 41.4s	remaining: 1m 25s
326:	learn: 1.3083995	total: 41.6s	remaining: 1m 25s
327:	learn: 1.3050350	total: 41.7s	remaining: 1m 25s
328:	learn: 1.3043622	total: 41.8s	remaining: 1m 25s
329:	learn: 1.3039804	total: 41.9s	remaining: 1m 25s
330:	learn: 1.3016892	total: 42.1s	remaining: 1m 25s
331:	learn: 1.2997752	total: 42.2s	remaining: 1m 24s
332:	learn: 1.2977507	total: 42.3s	remaining: 1m 24s
333:	learn: 1.2951227	total: 42.5s	remaining: 

471:	learn: 1.0955139	total: 1m	remaining: 1m 7s
472:	learn: 1.0939214	total: 1m	remaining: 1m 7s
473:	learn: 1.0920048	total: 1m	remaining: 1m 6s
474:	learn: 1.0898932	total: 1m	remaining: 1m 6s
475:	learn: 1.0888130	total: 1m	remaining: 1m 6s
476:	learn: 1.0869153	total: 1m	remaining: 1m 6s
477:	learn: 1.0861676	total: 1m	remaining: 1m 6s
478:	learn: 1.0859432	total: 1m	remaining: 1m 6s
479:	learn: 1.0857175	total: 1m 1s	remaining: 1m 6s
480:	learn: 1.0839209	total: 1m 1s	remaining: 1m 5s
481:	learn: 1.0816050	total: 1m 1s	remaining: 1m 5s
482:	learn: 1.0796631	total: 1m 1s	remaining: 1m 5s
483:	learn: 1.0789883	total: 1m 1s	remaining: 1m 5s
484:	learn: 1.0783816	total: 1m 1s	remaining: 1m 5s
485:	learn: 1.0771590	total: 1m 1s	remaining: 1m 5s
486:	learn: 1.0756003	total: 1m 1s	remaining: 1m 5s
487:	learn: 1.0753690	total: 1m 2s	remaining: 1m 5s
488:	learn: 1.0747116	total: 1m 2s	remaining: 1m 4s
489:	learn: 1.0723056	total: 1m 2s	remaining: 1m 4s
490:	learn: 1.0701390	total: 1m 2s	r

629:	learn: 0.9119101	total: 1m 20s	remaining: 47.1s
630:	learn: 0.9110731	total: 1m 20s	remaining: 46.9s
631:	learn: 0.9107389	total: 1m 20s	remaining: 46.8s
632:	learn: 0.9093849	total: 1m 20s	remaining: 46.7s
633:	learn: 0.9074709	total: 1m 20s	remaining: 46.6s
634:	learn: 0.9069542	total: 1m 20s	remaining: 46.4s
635:	learn: 0.9063690	total: 1m 20s	remaining: 46.3s
636:	learn: 0.9055931	total: 1m 21s	remaining: 46.2s
637:	learn: 0.9043679	total: 1m 21s	remaining: 46.1s
638:	learn: 0.9039279	total: 1m 21s	remaining: 45.9s
639:	learn: 0.9024132	total: 1m 21s	remaining: 45.8s
640:	learn: 0.9020772	total: 1m 21s	remaining: 45.7s
641:	learn: 0.9018223	total: 1m 21s	remaining: 45.5s
642:	learn: 0.9009362	total: 1m 21s	remaining: 45.4s
643:	learn: 0.9007840	total: 1m 21s	remaining: 45.3s
644:	learn: 0.9002969	total: 1m 22s	remaining: 45.2s
645:	learn: 0.8983005	total: 1m 22s	remaining: 45s
646:	learn: 0.8974068	total: 1m 22s	remaining: 44.9s
647:	learn: 0.8960207	total: 1m 22s	remaining: 4

...

243:	learn: 1.4758098	total: 31.1s	remaining: 1m 36s
244:	learn: 1.4724240	total: 31.2s	remaining: 1m 36s
245:	learn: 1.4711675	total: 31.3s	remaining: 1m 36s
246:	learn: 1.4706638	total: 31.4s	remaining: 1m 35s
247:	learn: 1.4696529	total: 31.6s	remaining: 1m 35s
248:	learn: 1.4671357	total: 31.7s	remaining: 1m 35s
249:	learn: 1.4630909	total: 31.8s	remaining: 1m 35s
250:	learn: 1.4597309	total: 32s	remaining: 1m 35s
251:	learn: 1.4565032	total: 32.1s	remaining: 1m 35s
252:	learn: 1.4532568	total: 32.2s	remaining: 1m 35s
253:	learn: 1.4506055	total: 32.3s	remaining: 1m 34s
254:	learn: 1.4483016	total: 32.5s	remaining: 1m 34s
255:	learn: 1.4446645	total: 32.6s	remaining: 1m 34s
256:	learn: 1.4420609	total: 32.7s	remaining: 1m 34s
257:	learn: 1.4395739	total: 32.9s	remaining: 1m 34s
258:	learn: 1.4381592	total: 33s	remaining: 1m 34s
259:	learn: 1.4362430	total: 33.1s	remaining: 1m 34s
260:	learn: 1.4353329	total: 33.2s	remaining: 1m 34s
261:	learn: 1.4346967	total: 33.4s	remaining: 1m 3

399:	learn: 1.1822659	total: 51s	remaining: 1m 16s
400:	learn: 1.1806735	total: 51.1s	remaining: 1m 16s
401:	learn: 1.1780738	total: 51.2s	remaining: 1m 16s
402:	learn: 1.1774990	total: 51.4s	remaining: 1m 16s
403:	learn: 1.1748325	total: 51.5s	remaining: 1m 15s
404:	learn: 1.1729591	total: 51.6s	remaining: 1m 15s
405:	learn: 1.1724534	total: 51.8s	remaining: 1m 15s
406:	learn: 1.1719605	total: 51.9s	remaining: 1m 15s
407:	learn: 1.1713853	total: 52s	remaining: 1m 15s
408:	learn: 1.1710686	total: 52.1s	remaining: 1m 15s
409:	learn: 1.1683615	total: 52.3s	remaining: 1m 15s
410:	learn: 1.1679053	total: 52.4s	remaining: 1m 15s
411:	learn: 1.1663571	total: 52.5s	remaining: 1m 14s
412:	learn: 1.1650534	total: 52.6s	remaining: 1m 14s
413:	learn: 1.1639716	total: 52.8s	remaining: 1m 14s
414:	learn: 1.1621392	total: 52.9s	remaining: 1m 14s
415:	learn: 1.1604534	total: 53s	remaining: 1m 14s
416:	learn: 1.1596385	total: 53.1s	remaining: 1m 14s
417:	learn: 1.1581133	total: 53.3s	remaining: 1m 14s

557:	learn: 0.9812441	total: 1m 11s	remaining: 56.3s
558:	learn: 0.9803793	total: 1m 11s	remaining: 56.2s
559:	learn: 0.9790662	total: 1m 11s	remaining: 56.1s
560:	learn: 0.9781589	total: 1m 11s	remaining: 55.9s
561:	learn: 0.9776141	total: 1m 11s	remaining: 55.8s
562:	learn: 0.9774263	total: 1m 11s	remaining: 55.7s
563:	learn: 0.9766697	total: 1m 11s	remaining: 55.6s
564:	learn: 0.9757190	total: 1m 11s	remaining: 55.4s
565:	learn: 0.9738672	total: 1m 12s	remaining: 55.3s
566:	learn: 0.9736463	total: 1m 12s	remaining: 55.2s
567:	learn: 0.9722921	total: 1m 12s	remaining: 55s
568:	learn: 0.9709815	total: 1m 12s	remaining: 54.9s
569:	learn: 0.9703964	total: 1m 12s	remaining: 54.8s
570:	learn: 0.9695907	total: 1m 12s	remaining: 54.7s
571:	learn: 0.9673845	total: 1m 12s	remaining: 54.5s
572:	learn: 0.9651104	total: 1m 13s	remaining: 54.4s
573:	learn: 0.9645046	total: 1m 13s	remaining: 54.3s
574:	learn: 0.9630538	total: 1m 13s	remaining: 54.2s
575:	learn: 0.9618192	total: 1m 13s	remaining: 5

713:	learn: 0.8403614	total: 1m 31s	remaining: 36.5s
714:	learn: 0.8395072	total: 1m 31s	remaining: 36.3s
715:	learn: 0.8386168	total: 1m 31s	remaining: 36.2s
716:	learn: 0.8375725	total: 1m 31s	remaining: 36.1s
717:	learn: 0.8369382	total: 1m 31s	remaining: 35.9s
718:	learn: 0.8355768	total: 1m 31s	remaining: 35.8s
719:	learn: 0.8351487	total: 1m 31s	remaining: 35.7s
720:	learn: 0.8346293	total: 1m 31s	remaining: 35.6s
721:	learn: 0.8341419	total: 1m 32s	remaining: 35.4s
722:	learn: 0.8333397	total: 1m 32s	remaining: 35.3s
723:	learn: 0.8327735	total: 1m 32s	remaining: 35.2s
724:	learn: 0.8322963	total: 1m 32s	remaining: 35.1s
725:	learn: 0.8313092	total: 1m 32s	remaining: 34.9s
726:	learn: 0.8299443	total: 1m 32s	remaining: 34.8s
727:	learn: 0.8296705	total: 1m 32s	remaining: 34.7s
728:	learn: 0.8294407	total: 1m 32s	remaining: 34.5s
729:	learn: 0.8280873	total: 1m 33s	remaining: 34.4s
730:	learn: 0.8275424	total: 1m 33s	remaining: 34.3s
731:	learn: 0.8271408	total: 1m 33s	remaining:

869:	learn: 0.7325362	total: 1m 50s	remaining: 16.6s
870:	learn: 0.7323549	total: 1m 50s	remaining: 16.4s
871:	learn: 0.7318196	total: 1m 51s	remaining: 16.3s
872:	learn: 0.7311747	total: 1m 51s	remaining: 16.2s
873:	learn: 0.7299373	total: 1m 51s	remaining: 16.1s
874:	learn: 0.7298163	total: 1m 51s	remaining: 15.9s
875:	learn: 0.7296435	total: 1m 51s	remaining: 15.8s
876:	learn: 0.7286182	total: 1m 51s	remaining: 15.7s
877:	learn: 0.7279534	total: 1m 51s	remaining: 15.5s
878:	learn: 0.7278169	total: 1m 51s	remaining: 15.4s
879:	learn: 0.7268283	total: 1m 52s	remaining: 15.3s
880:	learn: 0.7263561	total: 1m 52s	remaining: 15.2s
881:	learn: 0.7256011	total: 1m 52s	remaining: 15s
882:	learn: 0.7247873	total: 1m 52s	remaining: 14.9s
883:	learn: 0.7238128	total: 1m 52s	remaining: 14.8s
884:	learn: 0.7234541	total: 1m 52s	remaining: 14.7s
885:	learn: 0.7227421	total: 1m 52s	remaining: 14.5s
886:	learn: 0.7221819	total: 1m 53s	remaining: 14.4s
887:	learn: 0.7219051	total: 1m 53s	remaining: 1

In [16]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.869072093441629
0.8759548341558439
0.881730893932132
0.8682081328315775
0.849555932415395
0.869910673873196
0.6629909772287615
0.675395816366967
0.6463013421801107
0.680607152651373
0.6418288980361825
0.6708216769091747


In [17]:
train_size = 8000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.056868
0:	learn: 3.0986577	total: 169ms	remaining: 2m 49s
1:	learn: 3.0360666	total: 302ms	remaining: 2m 30s
2:	learn: 2.9775746	total: 435ms	remaining: 2m 24s
3:	learn: 2.9244052	total: 567ms	remaining: 2m 21s
4:	learn: 2.8749807	total: 700ms	remaining: 2m 19s
5:	learn: 2.8312745	total: 834ms	remaining: 2m 18s
6:	learn: 2.7872732	total: 964ms	remaining: 2m 16s
7:	learn: 2.7531111	total: 1.1s	remaining: 2m 15s
8:	learn: 2.7148656	total: 1.23s	remaining: 2m 15s
9:	learn: 2.6798105	total: 1.36s	remaining: 2m 15s
10:	learn: 2.6479874	total: 1.5s	remaining: 2m 14s
11:	learn: 2.6157456	total: 1.63s	remaining: 2m 14s
12:	learn: 2.5862684	total: 1.76s	remaining: 2m 13s
13:	learn: 2.5539222	total: 1.89s	remaining: 2m 13s
14:	learn: 2.5268590	total: 2.03s	remaining: 2m 13s
15:	learn: 2.5029726	total: 2.16s	remaining: 2m 12s
16:	learn: 2.4780451	total: 2.29s	remaining: 2m 12s
17:	learn: 2.4543776	total: 2.43s	remaining: 2m 12s
18:	learn: 2.4332904	total: 2.56s	remaining: 2

159:	learn: 1.6848147	total: 21.1s	remaining: 1m 50s
160:	learn: 1.6838169	total: 21.3s	remaining: 1m 50s
161:	learn: 1.6803271	total: 21.4s	remaining: 1m 50s
162:	learn: 1.6766639	total: 21.5s	remaining: 1m 50s
163:	learn: 1.6749658	total: 21.7s	remaining: 1m 50s
164:	learn: 1.6704086	total: 21.8s	remaining: 1m 50s
165:	learn: 1.6655684	total: 21.9s	remaining: 1m 50s
166:	learn: 1.6618598	total: 22.1s	remaining: 1m 50s
167:	learn: 1.6589974	total: 22.2s	remaining: 1m 49s
168:	learn: 1.6573467	total: 22.3s	remaining: 1m 49s
169:	learn: 1.6541860	total: 22.5s	remaining: 1m 49s
170:	learn: 1.6531637	total: 22.6s	remaining: 1m 49s
171:	learn: 1.6495933	total: 22.7s	remaining: 1m 49s
172:	learn: 1.6438459	total: 22.9s	remaining: 1m 49s
173:	learn: 1.6407927	total: 23s	remaining: 1m 49s
174:	learn: 1.6380881	total: 23.1s	remaining: 1m 48s
175:	learn: 1.6365270	total: 23.2s	remaining: 1m 48s
176:	learn: 1.6355683	total: 23.4s	remaining: 1m 48s
177:	learn: 1.6316829	total: 23.5s	remaining: 1m

315:	learn: 1.3150630	total: 41.7s	remaining: 1m 30s
316:	learn: 1.3139042	total: 41.9s	remaining: 1m 30s
317:	learn: 1.3117639	total: 42s	remaining: 1m 30s
318:	learn: 1.3111696	total: 42.1s	remaining: 1m 29s
319:	learn: 1.3099708	total: 42.3s	remaining: 1m 29s
320:	learn: 1.3081850	total: 42.4s	remaining: 1m 29s
321:	learn: 1.3062785	total: 42.5s	remaining: 1m 29s
322:	learn: 1.3059440	total: 42.7s	remaining: 1m 29s
323:	learn: 1.3041911	total: 42.8s	remaining: 1m 29s
324:	learn: 1.3016372	total: 42.9s	remaining: 1m 29s
325:	learn: 1.2996675	total: 43s	remaining: 1m 29s
326:	learn: 1.2977746	total: 43.2s	remaining: 1m 28s
327:	learn: 1.2962148	total: 43.3s	remaining: 1m 28s
328:	learn: 1.2939958	total: 43.4s	remaining: 1m 28s
329:	learn: 1.2924268	total: 43.6s	remaining: 1m 28s
330:	learn: 1.2900544	total: 43.7s	remaining: 1m 28s
331:	learn: 1.2894382	total: 43.8s	remaining: 1m 28s
332:	learn: 1.2873204	total: 44s	remaining: 1m 28s
333:	learn: 1.2846276	total: 44.1s	remaining: 1m 27s

471:	learn: 1.1005740	total: 1m 2s	remaining: 1m 9s
472:	learn: 1.1002861	total: 1m 2s	remaining: 1m 9s
473:	learn: 1.0996382	total: 1m 2s	remaining: 1m 9s
474:	learn: 1.0988024	total: 1m 2s	remaining: 1m 9s
475:	learn: 1.0985900	total: 1m 2s	remaining: 1m 9s
476:	learn: 1.0972464	total: 1m 2s	remaining: 1m 9s
477:	learn: 1.0968960	total: 1m 3s	remaining: 1m 8s
478:	learn: 1.0952817	total: 1m 3s	remaining: 1m 8s
479:	learn: 1.0944172	total: 1m 3s	remaining: 1m 8s
480:	learn: 1.0935355	total: 1m 3s	remaining: 1m 8s
481:	learn: 1.0921226	total: 1m 3s	remaining: 1m 8s
482:	learn: 1.0908692	total: 1m 3s	remaining: 1m 8s
483:	learn: 1.0891910	total: 1m 3s	remaining: 1m 8s
484:	learn: 1.0876989	total: 1m 4s	remaining: 1m 7s
485:	learn: 1.0861637	total: 1m 4s	remaining: 1m 7s
486:	learn: 1.0859883	total: 1m 4s	remaining: 1m 7s
487:	learn: 1.0854716	total: 1m 4s	remaining: 1m 7s
488:	learn: 1.0837293	total: 1m 4s	remaining: 1m 7s
489:	learn: 1.0825909	total: 1m 4s	remaining: 1m 7s
490:	learn: 

629:	learn: 0.9362182	total: 1m 23s	remaining: 48.9s
630:	learn: 0.9355606	total: 1m 23s	remaining: 48.8s
631:	learn: 0.9346379	total: 1m 23s	remaining: 48.6s
632:	learn: 0.9339011	total: 1m 23s	remaining: 48.5s
633:	learn: 0.9334364	total: 1m 23s	remaining: 48.4s
634:	learn: 0.9331886	total: 1m 23s	remaining: 48.2s
635:	learn: 0.9315869	total: 1m 24s	remaining: 48.1s
636:	learn: 0.9308184	total: 1m 24s	remaining: 48s
637:	learn: 0.9295139	total: 1m 24s	remaining: 47.8s
638:	learn: 0.9292430	total: 1m 24s	remaining: 47.7s
639:	learn: 0.9284250	total: 1m 24s	remaining: 47.6s
640:	learn: 0.9275937	total: 1m 24s	remaining: 47.4s
641:	learn: 0.9265523	total: 1m 24s	remaining: 47.3s
642:	learn: 0.9248412	total: 1m 24s	remaining: 47.2s
643:	learn: 0.9238116	total: 1m 25s	remaining: 47s
644:	learn: 0.9227370	total: 1m 25s	remaining: 46.9s
645:	learn: 0.9222582	total: 1m 25s	remaining: 46.8s
646:	learn: 0.9216386	total: 1m 25s	remaining: 46.6s
647:	learn: 0.9203095	total: 1m 25s	remaining: 46.

...

245:	learn: 1.4570536	total: 32.5s	remaining: 1m 39s
246:	learn: 1.4538076	total: 32.6s	remaining: 1m 39s
247:	learn: 1.4528931	total: 32.8s	remaining: 1m 39s
248:	learn: 1.4496256	total: 32.9s	remaining: 1m 39s
249:	learn: 1.4469442	total: 33s	remaining: 1m 39s
250:	learn: 1.4464883	total: 33.2s	remaining: 1m 38s
251:	learn: 1.4452909	total: 33.3s	remaining: 1m 38s
252:	learn: 1.4419473	total: 33.4s	remaining: 1m 38s
253:	learn: 1.4406908	total: 33.5s	remaining: 1m 38s
254:	learn: 1.4378766	total: 33.7s	remaining: 1m 38s
255:	learn: 1.4353331	total: 33.8s	remaining: 1m 38s
256:	learn: 1.4321918	total: 33.9s	remaining: 1m 38s
257:	learn: 1.4312833	total: 34.1s	remaining: 1m 38s
258:	learn: 1.4293456	total: 34.2s	remaining: 1m 37s
259:	learn: 1.4279206	total: 34.3s	remaining: 1m 37s
260:	learn: 1.4274820	total: 34.5s	remaining: 1m 37s
261:	learn: 1.4265774	total: 34.6s	remaining: 1m 37s
262:	learn: 1.4242346	total: 34.7s	remaining: 1m 37s
263:	learn: 1.4214477	total: 34.9s	remaining: 1m

401:	learn: 1.1878730	total: 53.1s	remaining: 1m 19s
402:	learn: 1.1862624	total: 53.3s	remaining: 1m 18s
403:	learn: 1.1847977	total: 53.4s	remaining: 1m 18s
404:	learn: 1.1832396	total: 53.5s	remaining: 1m 18s
405:	learn: 1.1824757	total: 53.7s	remaining: 1m 18s
406:	learn: 1.1801458	total: 53.8s	remaining: 1m 18s
407:	learn: 1.1788559	total: 53.9s	remaining: 1m 18s
408:	learn: 1.1772430	total: 54.1s	remaining: 1m 18s
409:	learn: 1.1760659	total: 54.2s	remaining: 1m 17s
410:	learn: 1.1755331	total: 54.3s	remaining: 1m 17s
411:	learn: 1.1726830	total: 54.4s	remaining: 1m 17s
412:	learn: 1.1711172	total: 54.6s	remaining: 1m 17s
413:	learn: 1.1694070	total: 54.7s	remaining: 1m 17s
414:	learn: 1.1676099	total: 54.8s	remaining: 1m 17s
415:	learn: 1.1654072	total: 55s	remaining: 1m 17s
416:	learn: 1.1643025	total: 55.1s	remaining: 1m 17s
417:	learn: 1.1628279	total: 55.2s	remaining: 1m 16s
418:	learn: 1.1614710	total: 55.4s	remaining: 1m 16s
419:	learn: 1.1609633	total: 55.5s	remaining: 1m

559:	learn: 0.9881521	total: 1m 14s	remaining: 58.2s
560:	learn: 0.9870863	total: 1m 14s	remaining: 58.1s
561:	learn: 0.9856933	total: 1m 14s	remaining: 57.9s
562:	learn: 0.9844984	total: 1m 14s	remaining: 57.8s
563:	learn: 0.9831064	total: 1m 14s	remaining: 57.7s
564:	learn: 0.9821385	total: 1m 14s	remaining: 57.5s
565:	learn: 0.9813965	total: 1m 14s	remaining: 57.4s
566:	learn: 0.9804412	total: 1m 14s	remaining: 57.3s
567:	learn: 0.9801655	total: 1m 15s	remaining: 57.1s
568:	learn: 0.9794185	total: 1m 15s	remaining: 57s
569:	learn: 0.9792475	total: 1m 15s	remaining: 56.9s
570:	learn: 0.9778770	total: 1m 15s	remaining: 56.7s
571:	learn: 0.9767387	total: 1m 15s	remaining: 56.6s
572:	learn: 0.9756972	total: 1m 15s	remaining: 56.5s
573:	learn: 0.9741099	total: 1m 15s	remaining: 56.3s
574:	learn: 0.9739435	total: 1m 16s	remaining: 56.2s
575:	learn: 0.9725390	total: 1m 16s	remaining: 56.1s
576:	learn: 0.9716305	total: 1m 16s	remaining: 56s
577:	learn: 0.9706175	total: 1m 16s	remaining: 55.

715:	learn: 0.8620836	total: 1m 34s	remaining: 37.6s
716:	learn: 0.8609356	total: 1m 34s	remaining: 37.4s
717:	learn: 0.8600941	total: 1m 34s	remaining: 37.3s
718:	learn: 0.8599854	total: 1m 35s	remaining: 37.2s
719:	learn: 0.8593186	total: 1m 35s	remaining: 37s
720:	learn: 0.8583404	total: 1m 35s	remaining: 36.9s
721:	learn: 0.8581141	total: 1m 35s	remaining: 36.8s
722:	learn: 0.8573296	total: 1m 35s	remaining: 36.6s
723:	learn: 0.8555183	total: 1m 35s	remaining: 36.5s
724:	learn: 0.8543621	total: 1m 35s	remaining: 36.4s
725:	learn: 0.8534681	total: 1m 36s	remaining: 36.2s
726:	learn: 0.8527149	total: 1m 36s	remaining: 36.1s
727:	learn: 0.8513659	total: 1m 36s	remaining: 36s
728:	learn: 0.8504912	total: 1m 36s	remaining: 35.8s
729:	learn: 0.8501847	total: 1m 36s	remaining: 35.7s
730:	learn: 0.8492170	total: 1m 36s	remaining: 35.6s
731:	learn: 0.8486590	total: 1m 36s	remaining: 35.4s
732:	learn: 0.8479570	total: 1m 36s	remaining: 35.3s
733:	learn: 0.8470725	total: 1m 37s	remaining: 35.

871:	learn: 0.7559939	total: 1m 55s	remaining: 16.9s
872:	learn: 0.7557104	total: 1m 55s	remaining: 16.8s
873:	learn: 0.7555308	total: 1m 55s	remaining: 16.7s
874:	learn: 0.7546251	total: 1m 55s	remaining: 16.5s
875:	learn: 0.7543858	total: 1m 55s	remaining: 16.4s
876:	learn: 0.7539091	total: 1m 55s	remaining: 16.3s
877:	learn: 0.7533265	total: 1m 56s	remaining: 16.1s
878:	learn: 0.7526138	total: 1m 56s	remaining: 16s
879:	learn: 0.7521858	total: 1m 56s	remaining: 15.9s
880:	learn: 0.7520431	total: 1m 56s	remaining: 15.7s
881:	learn: 0.7514959	total: 1m 56s	remaining: 15.6s
882:	learn: 0.7509341	total: 1m 56s	remaining: 15.5s
883:	learn: 0.7500677	total: 1m 56s	remaining: 15.3s
884:	learn: 0.7493351	total: 1m 57s	remaining: 15.2s
885:	learn: 0.7489749	total: 1m 57s	remaining: 15.1s
886:	learn: 0.7479077	total: 1m 57s	remaining: 14.9s
887:	learn: 0.7478024	total: 1m 57s	remaining: 14.8s
888:	learn: 0.7474515	total: 1m 57s	remaining: 14.7s
889:	learn: 0.7465597	total: 1m 57s	remaining: 1

In [18]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.8706836066922851
0.9146018380595767
0.8353963088118711
0.8781842697669859
0.8666028927295795
0.8586327240934116
0.660381654585536
0.6682717339400932
0.653586157480527
0.6682544830406274
0.6459053384067173
0.6658905600597156


In [19]:
train_size = 9000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.057936
0:	learn: 3.2152082	total: 167ms	remaining: 2m 46s
1:	learn: 3.1507356	total: 305ms	remaining: 2m 31s
2:	learn: 3.0919687	total: 442ms	remaining: 2m 26s
3:	learn: 3.0319517	total: 581ms	remaining: 2m 24s
4:	learn: 2.9790626	total: 720ms	remaining: 2m 23s
5:	learn: 2.9313763	total: 858ms	remaining: 2m 22s
6:	learn: 2.8801000	total: 998ms	remaining: 2m 21s
7:	learn: 2.8329762	total: 1.14s	remaining: 2m 20s
8:	learn: 2.7891917	total: 1.27s	remaining: 2m 20s
9:	learn: 2.7516228	total: 1.41s	remaining: 2m 19s
10:	learn: 2.7126563	total: 1.55s	remaining: 2m 19s
11:	learn: 2.6771214	total: 1.69s	remaining: 2m 18s
12:	learn: 2.6436124	total: 1.82s	remaining: 2m 18s
13:	learn: 2.6139961	total: 1.96s	remaining: 2m 18s
14:	learn: 2.5836964	total: 2.1s	remaining: 2m 18s
15:	learn: 2.5541109	total: 2.24s	remaining: 2m 17s
16:	learn: 2.5255757	total: 2.38s	remaining: 2m 17s
17:	learn: 2.5027006	total: 2.52s	remaining: 2m 17s
18:	learn: 2.4810884	total: 2.65s	remaining: 

159:	learn: 1.7000894	total: 21.9s	remaining: 1m 55s
160:	learn: 1.6945705	total: 22.1s	remaining: 1m 54s
161:	learn: 1.6934049	total: 22.2s	remaining: 1m 54s
162:	learn: 1.6895980	total: 22.3s	remaining: 1m 54s
163:	learn: 1.6880532	total: 22.5s	remaining: 1m 54s
164:	learn: 1.6844670	total: 22.6s	remaining: 1m 54s
165:	learn: 1.6836408	total: 22.7s	remaining: 1m 54s
166:	learn: 1.6816229	total: 22.9s	remaining: 1m 54s
167:	learn: 1.6804460	total: 23s	remaining: 1m 53s
168:	learn: 1.6767521	total: 23.1s	remaining: 1m 53s
169:	learn: 1.6748604	total: 23.3s	remaining: 1m 53s
170:	learn: 1.6714117	total: 23.4s	remaining: 1m 53s
171:	learn: 1.6673380	total: 23.6s	remaining: 1m 53s
172:	learn: 1.6647123	total: 23.7s	remaining: 1m 53s
173:	learn: 1.6634573	total: 23.8s	remaining: 1m 53s
174:	learn: 1.6620265	total: 24s	remaining: 1m 52s
175:	learn: 1.6612366	total: 24.1s	remaining: 1m 52s
176:	learn: 1.6573560	total: 24.2s	remaining: 1m 52s
177:	learn: 1.6542518	total: 24.4s	remaining: 1m 5

315:	learn: 1.3378687	total: 43.3s	remaining: 1m 33s
316:	learn: 1.3348913	total: 43.4s	remaining: 1m 33s
317:	learn: 1.3344822	total: 43.6s	remaining: 1m 33s
318:	learn: 1.3323177	total: 43.7s	remaining: 1m 33s
319:	learn: 1.3314050	total: 43.8s	remaining: 1m 33s
320:	learn: 1.3291658	total: 44s	remaining: 1m 33s
321:	learn: 1.3288523	total: 44.1s	remaining: 1m 32s
322:	learn: 1.3256705	total: 44.2s	remaining: 1m 32s
323:	learn: 1.3238715	total: 44.4s	remaining: 1m 32s
324:	learn: 1.3223644	total: 44.5s	remaining: 1m 32s
325:	learn: 1.3203419	total: 44.7s	remaining: 1m 32s
326:	learn: 1.3193774	total: 44.8s	remaining: 1m 32s
327:	learn: 1.3175805	total: 44.9s	remaining: 1m 32s
328:	learn: 1.3166123	total: 45.1s	remaining: 1m 31s
329:	learn: 1.3147665	total: 45.2s	remaining: 1m 31s
330:	learn: 1.3121452	total: 45.3s	remaining: 1m 31s
331:	learn: 1.3115953	total: 45.5s	remaining: 1m 31s
332:	learn: 1.3089562	total: 45.6s	remaining: 1m 31s
333:	learn: 1.3086298	total: 45.8s	remaining: 1m

471:	learn: 1.1218640	total: 1m 4s	remaining: 1m 12s
472:	learn: 1.1202830	total: 1m 4s	remaining: 1m 12s
473:	learn: 1.1200088	total: 1m 4s	remaining: 1m 12s
474:	learn: 1.1190813	total: 1m 5s	remaining: 1m 11s
475:	learn: 1.1175805	total: 1m 5s	remaining: 1m 11s
476:	learn: 1.1157838	total: 1m 5s	remaining: 1m 11s
477:	learn: 1.1149178	total: 1m 5s	remaining: 1m 11s
478:	learn: 1.1138449	total: 1m 5s	remaining: 1m 11s
479:	learn: 1.1129891	total: 1m 5s	remaining: 1m 11s
480:	learn: 1.1114062	total: 1m 5s	remaining: 1m 11s
481:	learn: 1.1099809	total: 1m 6s	remaining: 1m 10s
482:	learn: 1.1082780	total: 1m 6s	remaining: 1m 10s
483:	learn: 1.1080824	total: 1m 6s	remaining: 1m 10s
484:	learn: 1.1074388	total: 1m 6s	remaining: 1m 10s
485:	learn: 1.1062408	total: 1m 6s	remaining: 1m 10s
486:	learn: 1.1054929	total: 1m 6s	remaining: 1m 10s
487:	learn: 1.1033132	total: 1m 6s	remaining: 1m 10s
488:	learn: 1.1023891	total: 1m 7s	remaining: 1m 10s
489:	learn: 1.1007993	total: 1m 7s	remaining: 

627:	learn: 0.9636993	total: 1m 26s	remaining: 51s
628:	learn: 0.9630645	total: 1m 26s	remaining: 50.8s
629:	learn: 0.9625079	total: 1m 26s	remaining: 50.7s
630:	learn: 0.9617335	total: 1m 26s	remaining: 50.6s
631:	learn: 0.9605789	total: 1m 26s	remaining: 50.4s
632:	learn: 0.9597350	total: 1m 26s	remaining: 50.3s
633:	learn: 0.9590166	total: 1m 26s	remaining: 50.1s
634:	learn: 0.9582121	total: 1m 27s	remaining: 50s
635:	learn: 0.9568964	total: 1m 27s	remaining: 49.9s
636:	learn: 0.9563099	total: 1m 27s	remaining: 49.7s
637:	learn: 0.9554930	total: 1m 27s	remaining: 49.6s
638:	learn: 0.9543947	total: 1m 27s	remaining: 49.5s
639:	learn: 0.9535145	total: 1m 27s	remaining: 49.3s
640:	learn: 0.9530562	total: 1m 27s	remaining: 49.2s
641:	learn: 0.9525760	total: 1m 27s	remaining: 49.1s
642:	learn: 0.9506509	total: 1m 28s	remaining: 48.9s
643:	learn: 0.9500950	total: 1m 28s	remaining: 48.8s
644:	learn: 0.9498840	total: 1m 28s	remaining: 48.6s
645:	learn: 0.9483532	total: 1m 28s	remaining: 48.

...

237:	learn: 1.5002041	total: 32.5s	remaining: 1m 44s
238:	learn: 1.4972093	total: 32.7s	remaining: 1m 44s
239:	learn: 1.4961941	total: 32.8s	remaining: 1m 43s
240:	learn: 1.4952412	total: 32.9s	remaining: 1m 43s
241:	learn: 1.4923301	total: 33.1s	remaining: 1m 43s
242:	learn: 1.4899443	total: 33.2s	remaining: 1m 43s
243:	learn: 1.4869492	total: 33.4s	remaining: 1m 43s
244:	learn: 1.4841760	total: 33.5s	remaining: 1m 43s
245:	learn: 1.4830296	total: 33.6s	remaining: 1m 43s
246:	learn: 1.4806575	total: 33.8s	remaining: 1m 42s
247:	learn: 1.4785506	total: 33.9s	remaining: 1m 42s
248:	learn: 1.4762965	total: 34s	remaining: 1m 42s
249:	learn: 1.4741171	total: 34.2s	remaining: 1m 42s
250:	learn: 1.4702573	total: 34.3s	remaining: 1m 42s
251:	learn: 1.4668725	total: 34.5s	remaining: 1m 42s
252:	learn: 1.4656722	total: 34.6s	remaining: 1m 42s
253:	learn: 1.4643205	total: 34.7s	remaining: 1m 41s
254:	learn: 1.4629538	total: 34.9s	remaining: 1m 41s
255:	learn: 1.4625100	total: 35s	remaining: 1m 4

393:	learn: 1.2310353	total: 53.9s	remaining: 1m 22s
394:	learn: 1.2303254	total: 54s	remaining: 1m 22s
395:	learn: 1.2300686	total: 54.2s	remaining: 1m 22s
396:	learn: 1.2273865	total: 54.3s	remaining: 1m 22s
397:	learn: 1.2255253	total: 54.4s	remaining: 1m 22s
398:	learn: 1.2250155	total: 54.6s	remaining: 1m 22s
399:	learn: 1.2224675	total: 54.7s	remaining: 1m 22s
400:	learn: 1.2207341	total: 54.9s	remaining: 1m 21s
401:	learn: 1.2192009	total: 55s	remaining: 1m 21s
402:	learn: 1.2178068	total: 55.1s	remaining: 1m 21s
403:	learn: 1.2160506	total: 55.3s	remaining: 1m 21s
404:	learn: 1.2158176	total: 55.4s	remaining: 1m 21s
405:	learn: 1.2139729	total: 55.5s	remaining: 1m 21s
406:	learn: 1.2121182	total: 55.7s	remaining: 1m 21s
407:	learn: 1.2115207	total: 55.8s	remaining: 1m 20s
408:	learn: 1.2101138	total: 55.9s	remaining: 1m 20s
409:	learn: 1.2089572	total: 56.1s	remaining: 1m 20s
410:	learn: 1.2067198	total: 56.2s	remaining: 1m 20s
411:	learn: 1.2050364	total: 56.4s	remaining: 1m 2

549:	learn: 1.0492697	total: 1m 15s	remaining: 1m 1s
550:	learn: 1.0474172	total: 1m 15s	remaining: 1m 1s
551:	learn: 1.0466206	total: 1m 15s	remaining: 1m 1s
552:	learn: 1.0453572	total: 1m 15s	remaining: 1m 1s
553:	learn: 1.0439832	total: 1m 15s	remaining: 1m 1s
554:	learn: 1.0433421	total: 1m 15s	remaining: 1m
555:	learn: 1.0431822	total: 1m 16s	remaining: 1m
556:	learn: 1.0418690	total: 1m 16s	remaining: 1m
557:	learn: 1.0403309	total: 1m 16s	remaining: 1m
558:	learn: 1.0397985	total: 1m 16s	remaining: 1m
559:	learn: 1.0386388	total: 1m 16s	remaining: 1m
560:	learn: 1.0379375	total: 1m 16s	remaining: 1m
561:	learn: 1.0365594	total: 1m 16s	remaining: 59.9s
562:	learn: 1.0359837	total: 1m 17s	remaining: 59.8s
563:	learn: 1.0357112	total: 1m 17s	remaining: 59.6s
564:	learn: 1.0338994	total: 1m 17s	remaining: 59.5s
565:	learn: 1.0325878	total: 1m 17s	remaining: 59.4s
566:	learn: 1.0323811	total: 1m 17s	remaining: 59.2s
567:	learn: 1.0306854	total: 1m 17s	remaining: 59.1s
568:	learn: 1.

705:	learn: 0.9076658	total: 1m 36s	remaining: 40.2s
706:	learn: 0.9075123	total: 1m 36s	remaining: 40.1s
707:	learn: 0.9069642	total: 1m 36s	remaining: 40s
708:	learn: 0.9062525	total: 1m 37s	remaining: 39.8s
709:	learn: 0.9056227	total: 1m 37s	remaining: 39.7s
710:	learn: 0.9053262	total: 1m 37s	remaining: 39.6s
711:	learn: 0.9052050	total: 1m 37s	remaining: 39.4s
712:	learn: 0.9041888	total: 1m 37s	remaining: 39.3s
713:	learn: 0.9038772	total: 1m 37s	remaining: 39.1s
714:	learn: 0.9035714	total: 1m 37s	remaining: 39s
715:	learn: 0.9029658	total: 1m 38s	remaining: 38.9s
716:	learn: 0.9019242	total: 1m 38s	remaining: 38.7s
717:	learn: 0.9004198	total: 1m 38s	remaining: 38.6s
718:	learn: 0.8991343	total: 1m 38s	remaining: 38.5s
719:	learn: 0.8976823	total: 1m 38s	remaining: 38.3s
720:	learn: 0.8966944	total: 1m 38s	remaining: 38.2s
721:	learn: 0.8957296	total: 1m 38s	remaining: 38.1s
722:	learn: 0.8954973	total: 1m 38s	remaining: 37.9s
723:	learn: 0.8950001	total: 1m 39s	remaining: 37.

861:	learn: 0.8009644	total: 1m 58s	remaining: 18.9s
862:	learn: 0.7999347	total: 1m 58s	remaining: 18.8s
863:	learn: 0.7997734	total: 1m 58s	remaining: 18.6s
864:	learn: 0.7992557	total: 1m 58s	remaining: 18.5s
865:	learn: 0.7987257	total: 1m 58s	remaining: 18.3s
866:	learn: 0.7980500	total: 1m 58s	remaining: 18.2s
867:	learn: 0.7974424	total: 1m 58s	remaining: 18.1s
868:	learn: 0.7967880	total: 1m 58s	remaining: 17.9s
869:	learn: 0.7956737	total: 1m 59s	remaining: 17.8s
870:	learn: 0.7955267	total: 1m 59s	remaining: 17.7s
871:	learn: 0.7945053	total: 1m 59s	remaining: 17.5s
872:	learn: 0.7940121	total: 1m 59s	remaining: 17.4s
873:	learn: 0.7937031	total: 1m 59s	remaining: 17.3s
874:	learn: 0.7929475	total: 1m 59s	remaining: 17.1s
875:	learn: 0.7917468	total: 1m 59s	remaining: 17s
876:	learn: 0.7913697	total: 2m	remaining: 16.8s
877:	learn: 0.7910820	total: 2m	remaining: 16.7s
878:	learn: 0.7905745	total: 2m	remaining: 16.6s
879:	learn: 0.7897780	total: 2m	remaining: 16.4s
880:	learn:

In [21]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

0.8601658861598791
0.9058728276842868
0.8211994030223184
0.8033787866348258
0.857394254897386
0.9129841585605789
0.6928987774403147
0.6852270824426012
0.7028322158703502
0.7133375082117892
0.6947362941018218
0.6683607865750113
